
# Performance Evaluation of CVaR-VQE 


ridefinire time to solution
in lidar loro trovano l'ottimo per ogni N numero di istanze parallele
carleo non-stoquastic QA con troyer

Noi geralizziamo facendo una funzione in 2D oppure numero di shots come parametro t in modo fa fare una famiglia di funzioni

Cosa vogliamo: resolurce to solution, moltiplico N_shot e N_step
ferromagnetico bias
mi servono alcuni for
- random su theta
- scalo su shots 
- scalo su step (maxiter)

alpha suggerito da loro (0.25)

valuto con RTS (TTS dove moltiplico per gli shots) non normalizzato per n_max

n.b. green quantum computing paper montangero


## Plot for $p=1$ 


In [ ]:
import numpy as np
from scipy.optimize import curve_fit
from math import ceil
from json import JSONEncoder
from matplotlib import pyplot as plt
from src.utils import collect_results, NumpyArrayEncoder

In [ ]:
qubits = np.arange(4, 22, 2, dtype=int)
circ_depth = 1

psucc, ts, shots, nfevs = collect_results(qubits, circ_depth)

In [ ]:
for i, (p,t) in enumerate(zip(psucc, ts)):
    try:
        last_zero = np.argwhere(np.isclose(p, 0.))
        print(f"N={4+i*2}, t={np.asarray(t)[np.asarray(p)==0.].max()}, shots={np.asarray(shots[i])[np.asarray(p)==0.].max()}, nfev={np.asarray(nfevs[i])[np.asarray(p)==0.].max()}")
    except ValueError:
        print(f"No zeros in succes probability for N={4+i*2}")
    p_t = np.vstack((t, p))
    p_t = p_t[:, np.argsort(p_t[0, :])]
    plt.plot(p_t[0], p_t[1], ".", label=f"N={4+i*2}")

    plt.xscale('log')
    plt.xlabel(r"$t$ (shots x iterations)")
    plt.ylabel(r"$P$")
    plt.legend()

In [ ]:
def running_mean(x, n):
    cumsum = np.cumsum(np.pad(x, int(ceil(n/2)), mode='reflect')) 
    return (cumsum[n:] - cumsum[:-n]) / float(n)

In [ ]:
for i, (p,t) in enumerate(zip(psucc, ts)):
    p_t = np.vstack((t, p))
    p_t = p_t[:, np.argsort(p_t[0, :])]
    len_window = 12
    #p_t[1] = np.convolve(p_t[1], np.ones(len_window) / len_window, mode='same')
    p_t[1] = running_mean(p_t[1], len_window)
    plt.plot(p_t[0][::5], p_t[1][::5], "o-", label=f"N={4+i*2}")

    plt.xscale('log')
    plt.xlabel(r"$t$ (shots x iterations)")
    plt.ylabel(r"$P$")
    plt.legend()
    plt.savefig("p_t_n4-18.png")

In [ ]:
for k, (p,s,nfev) in enumerate(zip(psucc, shots, nfevs)):
    len_iter = np.max(nfev)//2+1
    p = np.reshape(p, (-1,len_iter))
    s = np.reshape(s, (-1,len_iter))
    nfev = np.reshape(nfev, (-1,len_iter))

    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,6))
    for i in range(p.shape[0]):
        ax1.plot(nfev[i,:], p[i,:], '--o', label=s[i,0])
    for j in range(p.shape[1]):
        ax2.plot(s[:,j], p[:,j], '--s', label=nfev[0,j])

    ax1.set_xlabel('Iterations')
    ax1.set_ylabel(r'$p$(success)')
    #ax1.set_ylim(0,1.1)

    ax2.set_xlabel('Shots')
    ax2.set_xscale('log', base=2)
    ax2.set_ylabel(r'$p$(success)')
    #ax2.set_ylim(0,1.1)

    ax1.legend(title='Shots')
    ax2.legend(title='Iterations')

    ax1.set_title(f'N={4+k*2}')
    ax2.set_title(f'N={4+k*2}')

In [ ]:
def compute_r(p, p_d=0.99):
    rt = np.log(1-p_d)
    rt /= np.log(1-p)
    return rt

tts_list = []
for p,t in zip(psucc, ts):
    p = np.asarray(p)
    t = np.asarray(t)
    #print(p[:20],t.shape)
    mask =  p == 0.
    mask += p == 1. # mask += p == 1.
    #p[mask] = 0.00005
    p = np.delete(p, mask)
    t = np.delete(t, mask)

    rt = compute_r(p)
    tts_list.append(t*rt)

In [ ]:
for i, (tts,t) in enumerate(zip(tts_list, ts)):
    if 4+i*2 > 18:
        pass#continue
    t = np.asarray(t)
    mask = np.asarray(psucc[i]) == 0.
    mask += np.asarray(psucc[i]) == 1. # mask += np.asarray(psucc[i]) == 1.
    t = np.delete(t, mask)

    tts_t = np.vstack((t, tts))
    tts_t = tts_t[:, np.argsort(tts_t[0, :])]
    # roll-mean
    len_window = 2
    tts_t[1] = running_mean(tts_t[1], len_window)
    plt.plot(tts_t[0], tts_t[1], ".", label=f"N={4+i*2}")

    plt.xscale('log')
    plt.yscale('log')
    #plt.xlim(1.5,50000)
    #plt.ylim(10000, 500000)
    plt.xlabel(r"$t$ (shots x iterations)")
    plt.ylabel(r"$STS$")
    plt.legend()
    plt.savefig("STS-t_roll8.png")

In [ ]:
def fun_exp(x, a, b):
    return a*2**(b*x)

def fun_lin(x, a, b):
    return a + b*x

In [ ]:
pt_all = []
for i, (p,t) in enumerate(zip(psucc, ts)):
    p_t = np.vstack((t, p))
    p_t = p_t[:, np.argsort(p_t[0, :])]
    pt_all.append(p_t)

colors = ['lightgreen', 'limegreen', 'forestgreen', 'darkgreen','lightseagreen', 'cornflowerblue' , 'blue', 'navy']
min_psucc_lst = [0.99, 0.9, 0.75, 0.5, 0.25]
min_qubits = 4 
max_qubits = 20 
for min_psucc, color in zip(min_psucc_lst, colors):
    step_size = []
    for i, pt in enumerate(pt_all):
        mask = pt[1] >= min_psucc
        # check if the p is not reached
        if mask.sum() == 0:
            continue
        step_size.append(pt[0][mask][0])

    x_axes = 4 + 2*np.arange(len(step_size))

    popt, _ = curve_fit(fun_lin, x_axes, np.log2(step_size))
    plt.plot(x_axes, step_size, "o", label=f"p={min_psucc}", color=color)
    plt.plot(np.arange(min_qubits, max_qubits+1), fun_exp(np.arange(min_qubits, max_qubits+1), 2**popt[0], popt[1]), "--", label=f"a={2**popt[0]:.2f}, b={popt[1]:.2f}", color=color)

plt.yscale('log', base=2)
plt.ylabel(r"$t$ (shots x iterations)")
plt.xlabel(r"$N$")

plt.legend(loc="best")
plt.title(r"Fit with $a \cdot 2^{b \cdot N}$")

plt.savefig("t_succ.png")

## Plot for $p=2$

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
from math import ceil
from json import JSONEncoder
from matplotlib import pyplot as plt
from src.utils import collect_results, NumpyArrayEncoder

In [ ]:
qubits = np.arange(4, 22, 2, dtype=int)
circ_depth = 2

psucc, ts, shots, nfevs = collect_results(qubits, circ_depth)

In [ ]:
for i, (p,t) in enumerate(zip(psucc, ts)):
    try:
        last_zero = np.argwhere(np.isclose(p, 0.))
        print(f"N={4+i*2}, t={np.asarray(t)[np.asarray(p)==0.].max()}, shots={np.asarray(shots[i])[np.asarray(p)==0.].max()}, nfev={np.asarray(nfevs[i])[np.asarray(p)==0.].max()}")
    except ValueError:
        print(f"No zeros in succes probability for N={4+i*2}")
    p_t = np.vstack((t, p))
    p_t = p_t[:, np.argsort(p_t[0, :])]
    plt.plot(p_t[0], p_t[1], ".", label=f"N={4+i*2}")

    plt.xscale('log')
    plt.xlabel(r"$t$ (shots x iterations)")
    plt.ylabel(r"$P$")
    plt.legend()
    #plt.savefig("p_t_n4-18.png")

In [ ]:
def running_mean(x, n):
    cumsum = np.cumsum(np.pad(x, int(ceil(n/2)), mode='reflect')) 
    return (cumsum[n:] - cumsum[:-n]) / float(n)

In [ ]:
for i, (p,t) in enumerate(zip(psucc, ts)):
    p_t = np.vstack((t, p))
    p_t = p_t[:, np.argsort(p_t[0, :])]
    len_window = 12
    #p_t[1] = np.convolve(p_t[1], np.ones(len_window) / len_window, mode='same')
    p_t[1] = running_mean(p_t[1], len_window)
    plt.plot(p_t[0][::5], p_t[1][::5], "o-", label=f"N={4+i*2}")

    plt.xscale('log')
    plt.xlabel(r"$t$ (shots x iterations)")
    plt.ylabel(r"$P$")
    plt.legend()
    plt.savefig("p_t_n4-18.png")

In [ ]:
for k, (p,s,nfev) in enumerate(zip(psucc, shots, nfevs)):
    len_iter = np.max(nfev)//2+1

    p = np.reshape(p, (-1,len_iter))
    s = np.reshape(s, (-1,len_iter))
    nfev = np.reshape(nfev, (-1,len_iter))

    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,6))
    for i in range(p.shape[0]):
        ax1.plot(nfev[i,:], p[i,:], '--o', label=s[i,0])
    for j in range(p.shape[1]):
        ax2.plot(s[:,j], p[:,j], '--s', label=nfev[0,j])

    ax1.set_xlabel('Iterations')
    ax1.set_ylabel(r'$p$(success)')
    #ax1.set_ylim(0,1.1)

    ax2.set_xlabel('Shots')
    ax2.set_xscale('log', base=2)
    ax2.set_ylabel(r'$p$(success)')
    #ax2.set_ylim(0,1.1)

    ax1.legend(title='Shots')
    ax2.legend(title='Iterations')

    ax1.set_title(f'N={4+k*2}')
    ax2.set_title(f'N={4+k*2}')

    #plt.savefig(f'shots_iter_N{4+k*2}.png')

In [ ]:
def compute_r(p, p_d=0.99):
    rt = np.log(1-p_d)
    rt /= np.log(1-p)
    return rt

tts_list = []
for p,t in zip(psucc, ts):
    p = np.asarray(p)
    t = np.asarray(t)
    #print(p[:20],t.shape)
    mask =  p == 0.
    mask += p == 1. # mask += p == 1.
    #p[mask] = 0.00005
    p = np.delete(p, mask)
    t = np.delete(t, mask)

    rt = compute_r(p)
    tts_list.append(t*rt)

In [ ]:
for i, (tts,t) in enumerate(zip(tts_list, ts)):
    if 4+i*2 > 18:
        pass#continue
    t = np.asarray(t)
    mask = np.asarray(psucc[i]) == 0.
    mask += np.asarray(psucc[i]) == 1. # mask += np.asarray(psucc[i]) == 1.
    t = np.delete(t, mask)

    tts_t = np.vstack((t, tts))
    tts_t = tts_t[:, np.argsort(tts_t[0, :])]
    # roll-mean
    len_window = 8
    tts_t[1] = running_mean(tts_t[1], len_window)
    plt.plot(tts_t[0], tts_t[1], ".", label=f"N={4+i*2}")

    plt.xscale('log')
    plt.yscale('log')
    #plt.xlim(1.5,50000)
    #plt.ylim(10000, 500000)
    plt.xlabel(r"$t$ (shots x iterations)")
    plt.ylabel(r"$STS$")
    plt.legend()
    plt.savefig("STS-t_roll8.png")

In [ ]:
def fun_exp(x, a, b):
    return a*2**(b*x)

def fun_lin(x, a, b):
    return a + b*x

In [ ]:
def fun_exp(x, a, b):
    return a*2**(b*x)

def fun_lin(x, a, b):
    return a + b*x
    
pt_all = []
for i, (p,t) in enumerate(zip(psucc, ts)):
    p_t = np.vstack((t, p))
    p_t = p_t[:, np.argsort(p_t[0, :])]
    pt_all.append(p_t)

colors = ['lightgreen', 'limegreen', 'forestgreen', 'darkgreen','lightseagreen', 'cornflowerblue' , 'blue', 'navy']
min_psucc_lst = [0.99, 0.9, 0.75, 0.5, 0.25]
min_qubits = 4 
max_qubits = 20 
for min_psucc, color in zip(min_psucc_lst, colors):
    step_size = []
    for i, pt in enumerate(pt_all):
        mask = pt[1] >= min_psucc
        # check if the p is not reached
        if mask.sum() == 0:
            continue
        step_size.append(pt[0][mask][0])

    x_axes = 4 + 2*np.arange(len(step_size))

    popt, _ = curve_fit(fun_lin, x_axes, np.log2(step_size))
    plt.plot(x_axes, step_size, "o", label=f"p={min_psucc}", color=color)
    plt.plot(np.arange(min_qubits, max_qubits+1), fun_exp(np.arange(min_qubits, max_qubits+1), 2**popt[0], popt[1]), "--", label=f"a={2**popt[0]:.2f}, b={popt[1]:.2f}", color=color)

plt.yscale('log', base=2)
plt.ylabel(r"$t$ (shots x iterations)")
plt.xlabel(r"$N$")

plt.legend(loc="best")
plt.title(r"Fit with $a \cdot 2^{b \cdot N}$")

plt.savefig("t_succ.png")


## Simulation with $p=1,2$



In [ ]:
qubits = np.arange(4, 22, 2, dtype=int)

psucc_1, ts_1, shots_1, nfevs_1 = collect_results(qubits, 1)
psucc_2, ts_2, shots_2, nfevs_2 = collect_results(qubits, 2)

In [ ]:
colors = ['lightgreen','forestgreen','tab:green','darkgreen', 'lightseagreen', 'tab:cyan','cornflowerblue' , 'blue', 'navy']
plots1 = []
plots2 = []

for i, (p,t) in enumerate(zip(psucc_1, ts_1)):
    if 4+i*2 > 18:
        continue
    try:
        last_zero = np.argwhere(np.isclose(p, 0.))
        print(f"N={4+i*2}, p=1, t={np.asarray(t)[np.asarray(p)==0.].max()}, shots={np.asarray(shots_1[i])[np.asarray(p)==0.].max()}, nfev={np.asarray(nfevs_1[i])[np.asarray(p)==0.].max()}")
    except ValueError:
        print(f"No zeros in success probability for N={4+i*2}")
    # plot for each dimension
    p_t_1 = np.vstack((t, p))
    p_t_1 = p_t_1[:, np.argsort(p_t_1[0, :])]
    plot1, = plt.plot(p_t_1[0], p_t_1[1], ".", label=f"{4+i*2}", color=colors[i])
    plots1.append(plot1)

    try:
        last_zero = np.argwhere(np.isclose(psucc_2[i], 0.))
        print(f"N={4+i*2}, p=2, t={np.asarray(ts_2[i])[np.asarray(psucc_2[i])==0.].max()}, shots={np.asarray(shots_2[i])[np.asarray(psucc_2[i])==0.].max()}, nfev={np.asarray(nfevs_2[i])[np.asarray(psucc_2[i])==0.].max()}")
    except ValueError:
        print(f"No zeros in succes probability for N={4+i*2}")
    p_t_2 = np.vstack((ts_2[i], psucc_2[i]))
    p_t_2 = p_t_2[:, np.argsort(p_t_2[0, :])]
    plot2, = plt.plot(p_t_2[0]*2, p_t_2[1], "o", label=f"{4+i*2}", color=colors[i], fillstyle='none')
    plots2.append(plot2)

    plt.xscale('log')
    plt.xlabel(r"$t$ (shots x iterations x depth)")
    plt.ylabel(r"$P$")

# Create a legend for the first line.
first_legend = plt.legend(handles=plots1, loc='upper right')
# Add the legend manually to the current Axes.
ax = plt.gca().add_artist(first_legend)

# Create another legend for the second line.
plt.legend(handles=plots2, loc='upper left')

    #plt.savefig("p_t_n4-18.png")

In [ ]:
colors = ['lightgreen','forestgreen','tab:green','darkgreen', 'lightseagreen', 'tab:cyan','cornflowerblue' , 'blue', 'navy']
plots1 = []
plots2 = []

for i, (p,t) in enumerate(zip(psucc_1, ts_1)):
    if 4+i*2 > 18:
        continue
    p_t1 = np.vstack((t, p))
    p_t1 = p_t1[:, np.argsort(p_t1[0, :])]
    len_window = 12
    #p_t1[1] = np.convolve(p_t1[1], np.ones(len_window) / len_window, mode='same')
    p_t1[1] = running_mean(p_t1[1], len_window)
    plot1, = plt.plot(p_t1[0][::5], p_t1[1][::5], "o-", label=f"{4+i*2}", color=colors[i])
    plots1.append(plot1)

    p_t2 = np.vstack((ts_2[i], psucc_2[i]))
    p_t2 = p_t2[:, np.argsort(p_t2[0, :])]
    p_t2[1] = running_mean(p_t2[1], len_window)
    plot2, = plt.plot(p_t2[0][::5]*2, p_t2[1][::5], "o-", label=f"{4+i*2}", fillstyle='none', color=colors[i])
    plots2.append(plot2)

    plt.xscale('log')
    plt.xlabel(r"$t$ (shots x iterations x depth)")
    plt.ylabel(r"$P$")

# Create a legend for the first line.
first_legend = plt.legend(handles=plots1, loc='upper right', title=r'$p=1$')
# Add the legend manually to the current Axes.
ax = plt.gca().add_artist(first_legend)

# Create another legend for the second line.
plt.legend(handles=plots2, loc='upper left', title=r'$p=2$')

In [ ]:
def fun_exp(x, a, b):
    return a*2**(b*x)

def fun_lin(x, a, b):
    return a + b*x
    
pt_all = []
for i, (p,t) in enumerate(zip(psucc_1, ts_1)):
    p_t = np.vstack((t, p))
    p_t = p_t[:, np.argsort(p_t[0, :])]
    pt_all.append(p_t)
pt_all_2 = []
for i, (p,t) in enumerate(zip(psucc_2, ts_2)):
    p_t = np.vstack((t, p))
    p_t = p_t[:, np.argsort(p_t[0, :])]
    pt_all_2.append(p_t)


colors = ['lightgreen', 'limegreen', 'forestgreen', 'darkgreen','lightseagreen', 'cornflowerblue' , 'blue', 'navy']
min_psucc_lst = [0.99, 0.9, 0.75, 0.5, 0.25]
min_qubits = 4 
max_qubits = 20 
for min_psucc, color in zip(min_psucc_lst, colors):
    step_size = []
    for i, pt in enumerate(pt_all):
        mask = pt[1] >= min_psucc
        # check if the p is not reached
        if mask.sum() == 0:
            continue
        step_size.append(pt[0][mask][0])

    x_axes = 4 + 2*np.arange(len(step_size))

    popt, _ = curve_fit(fun_lin, x_axes, np.log2(step_size))
    plt.plot(x_axes, step_size, "o", label=f"p={min_psucc}", color=color)
    plt.plot(np.arange(min_qubits, max_qubits+1), fun_exp(np.arange(min_qubits, max_qubits+1), 2**popt[0], popt[1]), "--", label=f"a={2**popt[0]:.2f}, b={popt[1]:.2f}", color=color)

    step_size_2 = []
    for i, pt_2 in enumerate(pt_all_2):
        mask = pt_2[1] >= min_psucc
        # check if the p is not reached
        if mask.sum() == 0:
            continue
        step_size_2.append(pt_2[0][mask][0])
    x_axes_2 = 4 + 2*np.arange(len(step_size_2))
    popt, _ = curve_fit(fun_lin, x_axes_2, np.log2(step_size_2))
    plt.plot(x_axes_2, step_size_2, "o", label=f"p={min_psucc}", color=color, fillstyle='none')
    plt.plot(np.arange(min_qubits, max_qubits+1), fun_exp(np.arange(min_qubits, max_qubits+1), 2**popt[0], popt[1]), "-", label=f"a={2**popt[0]:.2f}, b={popt[1]:.2f}", color=color)

plt.yscale('log', base=2)
plt.ylabel(r"$t$ (shots x iterations)")
plt.xlabel(r"$N$")

#plt.legend(loc="best")
plt.title(r"Fit with $a \cdot 2^{b \cdot N}$")

#plt.savefig("t_succ.png")


## Simulation with noise $p=2$



In [ ]:
import numpy as np
from scipy.optimize import curve_fit
from math import ceil
from json import JSONEncoder
from matplotlib import pyplot as plt
from src.utils import collect_results, NumpyArrayEncoder
import json
import os

In [ ]:
qubits = [4,6,8,10,12]
p = 2
# load exact simulations
psucc, ts, shots, nfevs = collect_results(qubits, p)

In [ ]:
ts_noise = []
shots_noise = []
nfevs_noise = []
psucc_noise = []
for qubit in qubits:
    dir_path = f"results/N{qubit}/noise/"
    print(f"directory: {dir_path}")
    p = []
    t = []
    s = []
    it = []
    # for each number of qubits
    # we have several number of shots and iterations
    for filename in sorted(os.listdir(dir_path)):
        filename = dir_path + filename
        with open(filename, "r") as file:
            # print(filename)
            data = json.load(file)
        ever_found = []
        # for each shot and iteration param
        # we randomized the initial point
        # to estimate the right probability
        for run in data:
            ever_found.append(run["ever_found"])
        # compute p('found minimum')
        p.append(
            np.mean(np.asarray(ever_found, dtype=np.float128))
        )  # float128 to avoid to many zeros
        # maxiter*shots = actual number of iteration
        t.append(run["shots"] * run["nfev"])
        s.append(run["shots"])
        it.append(run["nfev"])
    # update list for each number of qubits
    ts_noise.append(t)
    psucc_noise.append(p)
    shots_noise.append(s)
    nfevs_noise.append(it)

In [ ]:
colors = ['lightgreen','forestgreen','tab:green','darkgreen', 'lightseagreen', 'tab:cyan','cornflowerblue' , 'blue', 'navy']
plots1 = []
plots2 = []

for i, (p,t) in enumerate(zip(psucc, ts)):
    if 4+i*2 > 18:
        continue
    try:
        last_zero = np.argwhere(np.isclose(p, 0.))
        print(f"N={4+i*2}, p=2, t={np.asarray(t)[np.asarray(p)==0.].max()}, shots={np.asarray(shots[i])[np.asarray(p)==0.].max()}, nfev={np.asarray(nfevs[i])[np.asarray(p)==0.].max()}")
    except ValueError:
        print(f"No zeros in success probability for N={4+i*2}")
    # plot for each dimension
    p_t = np.vstack((t, p))
    p_t = p_t[:, np.argsort(p_t[0, :])]
    plot1, = plt.plot(p_t[0]*2, p_t[1], ".", label=f"{4+i*2}", color=colors[i], alpha=0.8)
    plots1.append(plot1)

for i, (p,t) in enumerate(zip(psucc_noise, ts_noise)):
    try:
        last_zero = np.argwhere(np.isclose(psucc_noise[i], 0.))
        print(f"N={4+i*2}, p=2, t={np.asarray(ts_noise[i])[np.asarray(psucc_noise[i])==0.].max()}, shots={np.asarray(shots_noise[i])[np.asarray(psucc_noise[i])==0.].max()}, nfev={np.asarray(nfevs_noise[i])[np.asarray(psucc_noise[i])==0.].max()}")
    except ValueError:
        print(f"No zeros in succes probability for N={4+i*2}")
    p_t_noise = np.vstack((ts_noise[i], psucc_noise[i]))
    p_t_noise = p_t_noise[:, np.argsort(p_t_noise[0, :])]
    plot2, = plt.plot(p_t_noise[0]*2, p_t_noise[1], ".", label=f"{4+i*2}", color=colors[i], fillstyle='none')
    plots2.append(plot2)

    plt.xscale('log')
    plt.xlabel(r"$t$ (shots x iterations x depth)")
    plt.ylabel(r"$P$")

# Create a legend for the first line.
first_legend = plt.legend(handles=plots1, loc='lower right', title="Exact")
#Add the legend manually to the current Axes.
ax = plt.gca().add_artist(first_legend)

plt.title(r"$p=2$")
#Create another legend for the second line.
plt.legend(handles=plots2, loc='upper left', title=f"Noise\nT1=50\u03bcs\nT2=70\u03bcs")

plt.savefig("noise_exact.png")

In [ ]:
colors = ['lightgreen','tab:green','darkgreen', 'lightseagreen','cornflowerblue' , 'blue', 'navy']
plots1 = []
plots2 = []

def running_mean(x, n):
    cumsum = np.cumsum(np.pad(x, int(ceil(n/2)), mode='reflect')) 
    return (cumsum[n:] - cumsum[:-n]) / float(n)

for i, (p,t) in enumerate(zip(psucc, ts)):
    if 4+i*2 > 18:
        continue
    try:
        last_zero = np.argwhere(np.isclose(p, 0.))
        print(f"N={4+i*2}, p=2, t={np.asarray(t)[np.asarray(p)==0.].max()}, shots={np.asarray(shots[i])[np.asarray(p)==0.].max()}, nfev={np.asarray(nfevs[i])[np.asarray(p)==0.].max()}")
    except ValueError:
        print(f"No zeros in success probability for N={4+i*2}")
    # plot for each dimension
    p_t = np.vstack((t, p))
    p_t = p_t[:, np.argsort(p_t[0, :])]
    len_window = 12
    p_t[1] = running_mean(p_t[1], len_window)
    plot1, = plt.plot(p_t[0][::5]*2, p_t[1][::5], "-o", label=f"{4+i*2}", color=colors[i], alpha=0.8)
    plt.vlines(2**(4+i*2), 0, 1, colors=colors[i], linestyles='dashed', alpha=0.6, linewidth=2.5)
    plots1.append(plot1)

for i, (p,t) in enumerate(zip(psucc_noise, ts_noise)):
    try:
        last_zero = np.argwhere(np.isclose(psucc_noise[i], 0.))
        print(f"N={4+i*2}, p=2, t={np.asarray(ts_noise[i])[np.asarray(psucc_noise[i])==0.].max()}, shots={np.asarray(shots_noise[i])[np.asarray(psucc_noise[i])==0.].max()}, nfev={np.asarray(nfevs_noise[i])[np.asarray(psucc_noise[i])==0.].max()}")
    except ValueError:
        print(f"No zeros in succes probability for N={4+i*2}")
    p_t_noise = np.vstack((ts_noise[i], psucc_noise[i]))
    p_t_noise = p_t_noise[:, np.argsort(p_t_noise[0, :])]
    len_window = 12
    p_t_noise[1] = running_mean(p_t_noise[1], len_window)
    plot2, = plt.plot(p_t_noise[0][::5]*2, p_t_noise[1][::5], "o-", label=f"{4+i*2}", color=colors[i], fillstyle='none')
    plots2.append(plot2)

    plt.xscale('log')
    plt.xlabel(r"$t$ (shots x iterations x depth)")
    plt.ylabel(r"$P$")

# Create a legend for the first line.
first_legend = plt.legend(handles=plots1, loc='lower right', title="Exact")
#Add the legend manually to the current Axes.
ax = plt.gca().add_artist(first_legend)

plt.title(r"$p=2$")
#Create another legend for the second line.
plt.legend(handles=plots2, loc='upper left', title=f"Noise\nT1=50\u03bcs\nT2=70\u03bcs")

plt.savefig("noise_exact_roll.png")

In [ ]:
def fun_exp(x, a, b):
    return a*2**(b*x)

def fun_lin(x, a, b):
    return a + b*x
    
pt_all = []
for i, (p,t) in enumerate(zip(psucc, ts)):
    p_t = np.vstack((t, p))
    p_t = p_t[:, np.argsort(p_t[0, :])]
    pt_all.append(p_t)
pt_all_2 = []
for i, (p,t) in enumerate(zip(psucc_noise, ts_noise)):
    p_t = np.vstack((t, p))
    p_t = p_t[:, np.argsort(p_t[0, :])]
    pt_all_2.append(p_t)


colors = ['lightgreen', 'limegreen', 'forestgreen', 'darkgreen','lightseagreen', 'cornflowerblue' , 'blue', 'navy']
min_psucc_lst = [0.99, 0.9, 0.5, 0.25]
min_qubits = 4 
max_qubits = 14
plots1 = [] 
plots2 = []
for min_psucc, color in zip(min_psucc_lst, colors[::2]):
    step_size = []
    for i, pt in enumerate(pt_all):
        mask = pt[1] >= min_psucc
        # check if the p is not reached
        if mask.sum() == 0:
            continue
        step_size.append(pt[0][mask][0])

    x_axes = 4 + 2*np.arange(len(step_size))

    popt, _ = curve_fit(fun_lin, x_axes, np.log2(step_size))
    plot1, = plt.plot(x_axes, step_size, "o", label=f"p={min_psucc}", color=color)
    plots1.append(plot1)
    plot1, = plt.plot(np.arange(min_qubits, max_qubits+1), fun_exp(np.arange(min_qubits, max_qubits+1), 2**popt[0], popt[1]), "-", label=f"a={2**popt[0]:.2f}, b={popt[1]:.2f}", color=color)
    plots1.append(plot1)

    step_size_2 = []
    for i, pt_2 in enumerate(pt_all_2):
        mask = pt_2[1] >= min_psucc
        # check if the p is not reached
        if mask.sum() == 0:
            continue
        step_size_2.append(pt_2[0][mask][0])
    x_axes_2 = 4 + 2*np.arange(len(step_size_2))
    popt, _ = curve_fit(fun_lin, x_axes_2, np.log2(step_size_2))
    plot2, = plt.plot(x_axes_2, step_size_2, "o", label=f"p={min_psucc}", color=color, fillstyle='none')
    plots2.append(plot2)
    plot2, = plt.plot(np.arange(min_qubits, max_qubits+1), fun_exp(np.arange(min_qubits, max_qubits+1), 2**popt[0], popt[1]), "--", label=f"a={2**popt[0]:.2f}, b={popt[1]:.2f}", color=color)
    plots2.append(plot2)

plt.yscale('log', base=2)
plt.ylabel(r"$t$ (shots x iterations)")
plt.xlabel(r"$N$")


# Create a legend for the first line.
first_legend = plt.legend(handles=plots1, loc='lower right', title="Exact", fontsize="x-small")
#Add the legend manually to the current Axes.
ax = plt.gca().add_artist(first_legend)

plt.title(r"$p=2$")
#Create another legend for the second line.
plt.legend(handles=plots2, loc='upper left', title="Noise\n" + r"$T_1=50$" + f"\u03bcs" r" $T_2=70$" + f"\u03bcs", fontsize="x-small")

plt.title(r"Fit with $a \cdot 2^{b \cdot N}$")

plt.savefig("t_scale.png")

In [ ]:
def fun_exp(x, a, b):
    return a*2**(x**b)

def fun_lin(x, a, b):
    return a + x**b
    
pt_all = []
for i, (p,t) in enumerate(zip(psucc, ts)):
    p_t = np.vstack((t, p))
    p_t = p_t[:, np.argsort(p_t[0, :])]
    pt_all.append(p_t)
pt_all_2 = []
for i, (p,t) in enumerate(zip(psucc_noise, ts_noise)):
    p_t = np.vstack((t, p))
    p_t = p_t[:, np.argsort(p_t[0, :])]
    pt_all_2.append(p_t)


colors = ['lightgreen', 'limegreen', 'forestgreen', 'darkgreen','lightseagreen', 'cornflowerblue' , 'blue', 'navy']
min_psucc_lst = [0.99, 0.9, 0.5, 0.25]
min_qubits = 4 
max_qubits = 20
plots1 = [] 
plots2 = []
for min_psucc, color in zip(min_psucc_lst, colors[::2]):
    step_size = []
    for i, pt in enumerate(pt_all):
        mask = pt[1] >= min_psucc
        # check if the p is not reached
        if mask.sum() == 0:
            continue
        step_size.append(pt[0][mask][0])

    x_axes = 4 + 2*np.arange(len(step_size))

    popt, _ = curve_fit(fun_lin, x_axes, np.log2(step_size))
    plot1, = plt.plot(x_axes, step_size, "o", label=f"p={min_psucc}", color=color)
    plots1.append(plot1)
    plot1, = plt.plot(np.arange(min_qubits, max_qubits+1), fun_exp(np.arange(min_qubits, max_qubits+1), 2**popt[0], popt[1]), "-", label=f"a={2**popt[0]:.2f}, b={popt[1]:.2f}", color=color)
    plots1.append(plot1)

    step_size_2 = []
    for i, pt_2 in enumerate(pt_all_2):
        mask = pt_2[1] >= min_psucc
        # check if the p is not reached
        if mask.sum() == 0:
            continue
        step_size_2.append(pt_2[0][mask][0])
    x_axes_2 = 4 + 2*np.arange(len(step_size_2))
    popt, _ = curve_fit(fun_lin, x_axes_2, np.log2(step_size_2))
    plot2, = plt.plot(x_axes_2, step_size_2, "o", label=f"p={min_psucc}", color=color, fillstyle='none')
    plots2.append(plot2)
    plot2, = plt.plot(np.arange(min_qubits, max_qubits+1), fun_exp(np.arange(min_qubits, max_qubits+1), 2**popt[0], popt[1]), "--", label=f"a={2**popt[0]:.2f}, b={popt[1]:.2f}", color=color)
    plots2.append(plot2)

plt.yscale('log', base=2)
plt.ylabel(r"$t$ (shots x iterations)")
plt.xlabel(r"$N$")


# Create a legend for the first line.
first_legend = plt.legend(handles=plots1, loc='lower right', title="Exact", fontsize="small")
#Add the legend manually to the current Axes.
ax = plt.gca().add_artist(first_legend)

plt.title(r"$p=2$")
#Create another legend for the second line.
plt.legend(handles=plots2, loc='upper left', title="Noise\n" + r"$T_1=50$" + f"\u03bcs" r" $T_2=70$" + f"\u03bcs", fontsize="small")

plt.title(r"Fit with $a \cdot 2^{N^b}$")

plt.savefig("t_scale_stretch.png")


## QAOA+ vs VQE

Custom implementation of a QAOA with an increased number of parameters, i.e., a number comparable to the parameters of a standard VQE, in order to make a fair comparison with VQE $p=2$.

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
from math import ceil
from json import JSONEncoder
from matplotlib import pyplot as plt
from src.utils import collect_results, NumpyArrayEncoder, create_ansatz
import json
import os
import pandas as pd

In [ ]:
qc = create_ansatz(qubits=4, circ_depth=2, ansatz_type='qaoa+')
#qc.draw() @uncomment if you want see the circuit

In [ ]:
qubits = [4,6,8,10,12,14,16,18]
p = 2
# load exact simulations
psucc_vqe, ts_vqe, shots_vqe, nfevs_vqe = collect_results(qubits, p)

In [ ]:
ts_qaoap = []
shots_qaoap = []
nfevs_qaoap = []
psucc_qaoap = []
for qubit in qubits:
    dir_path = f"results/N{qubit}/qaoa+/" if qubit < 14 else f"results/N{qubit}/qaoa+_iter80/"
    print(f"directory: {dir_path}")
    p = []
    t = []
    s = []
    it = []
    # for each number of qubits
    # we have several number of shots and iterations
    for filename in sorted(os.listdir(dir_path)):
        filename = dir_path + filename
        with open(filename, "r") as file:
            # print(filename)
            data = json.load(file)
        ever_found = []
        # for each shot and iteration param
        # we randomized the initial point
        # to estimate the right probability
        for run in data:
            ever_found.append(run["ever_found"])
        # compute p('found minimum')
        p.append(
            np.mean(np.asarray(ever_found, dtype=np.float128))
        )  # float128 to avoid to many zeros
        # maxiter*shots = actual number of iteration
        t.append(run["shots"] * run["nfev"])
        s.append(run["shots"])
        it.append(run["nfev"])
    # update list for each number of qubits
    ts_qaoap.append(t)
    psucc_qaoap.append(p)
    shots_qaoap.append(s)
    nfevs_qaoap.append(it)

In [ ]:
colors = ['lightgreen','forestgreen','tab:green','darkgreen', 'lightseagreen', 'tab:cyan','cornflowerblue' , 'blue', 'navy']
plots1 = []
plots2 = []

for i, (p,t) in enumerate(zip(psucc_vqe, ts_vqe)):
    if 4+i*2 > 18:
        continue
    try:
        last_zero = np.argwhere(np.isclose(p, 0.))
        print(f"N={4+i*2}, p=2, t={np.asarray(t)[np.asarray(p)==0.].max()}, shots={np.asarray(shots_vqe[i])[np.asarray(p)==0.].max()}, nfev={np.asarray(nfevs_vqe[i])[np.asarray(p)==0.].max()}")
    except ValueError:
        print(f"No zeros in success probability for N={4+i*2}")
    # plot for each dimension
    p_t = np.vstack((t, p))
    p_t = p_t[:, np.argsort(p_t[0, :])]
    plot1, = plt.plot(p_t[0]*2, p_t[1], ".", label=f"{4+i*2}", color=colors[i], alpha=0.8)
    plots1.append(plot1)

for i, (p,t) in enumerate(zip(psucc_qaoap, ts_qaoap)):
    try:
        last_zero = np.argwhere(np.isclose(psucc_qaoap[i], 0.))
        print(f"N={4+i*2}, p=2, t={np.asarray(ts_qaoap[i])[np.asarray(psucc_qaoap[i])==0.].max()}, shots={np.asarray(shots_qaoap[i])[np.asarray(psucc_qaoap[i])==0.].max()}, nfev={np.asarray(nfevs_qaoap[i])[np.asarray(psucc_qaoap[i])==0.].max()}")
    except ValueError:
        print(f"No zeros in succes probability for N={4+i*2}")
    p_t_qaoap = np.vstack((ts_qaoap[i], psucc_qaoap[i]))
    p_t_qaoap = p_t_qaoap[:, np.argsort(p_t_qaoap[0, :])]
    plot2, = plt.plot(p_t_qaoap[0]*2, p_t_qaoap[1], ".", label=f"{4+i*2}", color=colors[i], fillstyle='none')
    plots2.append(plot2)

    plt.xscale('log')
    plt.xlabel(r"$t$ (shots x iterations x depth)")
    plt.ylabel(r"$P$")

# Create a legend for the first line.
first_legend = plt.legend(handles=plots1, loc='lower right', title="VQE")
#Add the legend manually to the current Axes.
ax = plt.gca().add_artist(first_legend)

plt.title(r"$p=2$")
#Create another legend for the second line.
plt.legend(handles=plots2, loc='upper left', title=f"QAOA+")

#plt.savefig("qaoa_vqe.png")

In [ ]:
colors = ['lightgreen','tab:green','darkgreen', 'lightseagreen','cornflowerblue' , 'blue', 'navy']
plots1 = []
plots2 = []

def running_mean(x, n):
    cumsum = np.cumsum(np.pad(x, int(ceil(n/2)), mode='reflect')) 
    return (cumsum[n:] - cumsum[:-n]) / float(n)

for i, (p,t) in enumerate(zip(psucc_vqe, ts_vqe)):
    if 4+i*2 > 18:
        continue
    try:
        last_zero = np.argwhere(np.isclose(p, 0.))
        print(f"N={4+i*2}, p=2, t={np.asarray(t)[np.asarray(p)==0.].max()}, shots={np.asarray(shots_vqe[i])[np.asarray(p)==0.].max()}, nfev={np.asarray(nfevs_vqe[i])[np.asarray(p)==0.].max()}")
    except ValueError:
        print(f"No zeros in success probability for N={4+i*2}")
    # plot for each dimension
    p_t = np.vstack((t, p))
    p_t = p_t[:, np.argsort(p_t[0, :])]
    len_window = 12
    p_t[1] = running_mean(p_t[1], len_window)
    plot1, = plt.plot(p_t[0][::5]*2, p_t[1][::5], "-o", label=f"{4+i*2}", color=colors[i], alpha=0.8)
    plt.vlines(2**(4+i*2), 0, 1, colors=colors[i], linestyles='dashed', alpha=0.6, linewidth=2.5)
    plots1.append(plot1)

for i, (p,t) in enumerate(zip(psucc_qaoap, ts_qaoap)):
    try:
        last_zero = np.argwhere(np.isclose(psucc_qaoap[i], 0.))
        print(f"N={4+i*2}, p=2, t={np.asarray(ts_qaoap[i])[np.asarray(psucc_qaoap[i])==0.].max()}, shots={np.asarray(shots_qaoap[i])[np.asarray(psucc_qaoap[i])==0.].max()}, nfev={np.asarray(nfevs_qaoap[i])[np.asarray(psucc_qaoap[i])==0.].max()}")
    except ValueError:
        print(f"No zeros in succes probability for N={4+i*2}")
    p_t_qaoap = np.vstack((ts_qaoap[i], psucc_qaoap[i]))
    p_t_qaoap = p_t_qaoap[:, np.argsort(p_t_qaoap[0, :])]
    len_window = 12
    p_t_qaoap[1] = running_mean(p_t_qaoap[1], len_window)
    plot2, = plt.plot(p_t_qaoap[0][::5]*2, p_t_qaoap[1][::5], "o-", label=f"{4+i*2}", color=colors[i], fillstyle='none')
    plots2.append(plot2)

    plt.xscale('log')
    plt.xlabel(r"$t$ (shots x iterations x depth)")
    plt.ylabel(r"$P$")

# Create a legend for the first line.
first_legend = plt.legend(handles=plots1, loc='lower right', title="Exact")
#Add the legend manually to the current Axes.
ax = plt.gca().add_artist(first_legend)

plt.title(r"$p=2$")
#Create another legend for the second line.
plt.legend(handles=plots2, loc='upper left', title=f"QAOA+")

plt.savefig("noise_exact_roll.png")

In [ ]:
def fun_exp(x, a, b):
    return a*2**(b*x)

def fun_lin(x, a, b):
    return a + b*x
    
pt_all = []
for i, (p,t) in enumerate(zip(psucc_vqe, ts_vqe)):
    p_t = np.vstack((t, p))
    p_t = p_t[:, np.argsort(p_t[0, :])]
    pt_all.append(p_t)
pt_all_2 = []
for i, (p,t) in enumerate(zip(psucc_qaoap, ts_qaoap)):
    p_t = np.vstack((t, p))
    p_t = p_t[:, np.argsort(p_t[0, :])]
    pt_all_2.append(p_t)


colors = ['lightgreen', 'limegreen', 'forestgreen', 'darkgreen','lightseagreen', 'cornflowerblue' , 'blue', 'navy']
min_psucc_lst = [0.99, 0.9, 0.5, 0.25]
min_qubits = 4 
max_qubits = 16
plots1 = [] 
plots2 = []

plt.plot(np.arange(min_qubits, max_qubits+1), fun_exp(np.arange(min_qubits, max_qubits+1), 1, 1), color='red', alpha=0.7)

for min_psucc, color in zip(min_psucc_lst, colors[::2]):
    step_size = []
    for i, pt in enumerate(pt_all):
        mask = pt[1] >= min_psucc
        # check if the p is not reached
        if mask.sum() == 0:
            continue
        step_size.append(pt[0][mask][0]*2)

    x_axes = 4 + 2*np.arange(len(step_size))

    popt, _ = curve_fit(fun_lin, x_axes, np.log2(step_size))
    plot1, = plt.plot(x_axes, step_size, "o", label=f"p={min_psucc}", color=color)
    plots1.append(plot1)
    plot1, = plt.plot(np.arange(min_qubits, max_qubits+1), fun_exp(np.arange(min_qubits, max_qubits+1), 2**popt[0], popt[1]), "-", label=f"a={2**popt[0]:.2f}, b={popt[1]:.2f}", color=color)
    plots1.append(plot1)

    step_size_2 = []
    for i, pt_2 in enumerate(pt_all_2):
        mask = pt_2[1] >= min_psucc
        # check if the p is not reached
        if mask.sum() == 0:
            continue
        step_size_2.append(pt_2[0][mask][0]*2)
    x_axes_2 = 4 + 2*np.arange(len(step_size_2))
    popt, _ = curve_fit(fun_lin, x_axes_2, np.log2(step_size_2))
    plot2, = plt.plot(x_axes_2, step_size_2, "o", label=f"p={min_psucc}", color=color, fillstyle='none')
    plots2.append(plot2)
    plot2, = plt.plot(np.arange(min_qubits, max_qubits+1), fun_exp(np.arange(min_qubits, max_qubits+1), 2**popt[0], popt[1]), "--", label=f"a={2**popt[0]:.2f}, b={popt[1]:.2f}", color=color)
    plots2.append(plot2)

plt.yscale('log', base=2)
plt.ylabel(r"$t$ (shots x iterations)")
plt.xlabel(r"$N$")


# Create a legend for the first line.
first_legend = plt.legend(handles=plots1, loc='upper left', title="VQE", fontsize="x-small")
#Add the legend manually to the current Axes.
ax = plt.gca().add_artist(first_legend)

plt.title(r"$p=2$")
#Create another legend for the second line.
plt.legend(handles=plots2, loc='lower right', title="QAOA+", fontsize="x-small")

plt.title(r"Fit with $a \cdot 2^{b \cdot N}$")

#plt.savefig("t_scale.png")


### Comparison QAOA+ and VQE for iteration
 

In [ ]:
for k, (p_qaoa, s_qaoa, nfev_qaoa, p_vqe, s_vqe, nfev_vqe) in enumerate(zip(psucc_qaoap, shots_qaoap, nfevs_qaoap, psucc_vqe, shots_vqe, nfevs_vqe)):
    len_iter_qaoa = np.max(nfev_qaoa)//2+1
    p_qaoa = np.reshape(p_qaoa, (-1,len_iter_qaoa))
    s_qaoa = np.reshape(s_qaoa, (-1,len_iter_qaoa))
    nfev_qaoa = np.reshape(nfev_qaoa, (-1,len_iter_qaoa))

    len_iter_vqe = np.max(nfev_vqe)//2+1
    p_vqe = np.reshape(p_vqe, (-1,len_iter_vqe))
    s_vqe = np.reshape(s_vqe, (-1,len_iter_vqe))
    nfev_vqe = np.reshape(nfev_vqe, (-1,len_iter_vqe))

    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,6))
    for i in range(p_qaoa.shape[0]):
        ax1.plot(nfev_qaoa[i,:], p_qaoa[i,:], '--o', label=s_qaoa[i,0])
    for i in range(p_vqe.shape[0]):
        ax2.plot(nfev_vqe[i,:], p_vqe[i,:], '--o', label=s_vqe[i,0])

    ax1.set_xlabel('Iterations')
    ax1.set_ylabel(r'$F(success \leq t)$')
    #ax1.set_ylim(0,1.1)

    ax2.set_xlabel('Iterations')
    ax2.set_ylabel(r'$F(success \leq t)$')
    #ax2.set_ylim(0,1.1)

    ax1.legend(title='Shots')
    ax2.legend(title='Shots')

    ax1.set_title(f'QAOA+ N={4+k*2}')
    ax2.set_title(f'VQE N={4+k*2}')


### Comparison QAOA+ and VQE for shots


In [ ]:
for k, (p_qaoa, s_qaoa, nfev_qaoa, t_qaoa, p_vqe, s_vqe, nfev_vqe, t_vqe) in enumerate(zip(psucc_qaoap, shots_qaoap, nfevs_qaoap, ts_qaoap, psucc_vqe, shots_vqe, nfevs_vqe, ts_vqe)):
    len_iter_qaoa = np.max(nfev_qaoa)//2+1
    p_qaoa = np.reshape(p_qaoa, (-1,len_iter_qaoa))
    s_qaoa = np.reshape(s_qaoa, (-1,len_iter_qaoa))
    nfev_qaoa = np.reshape(nfev_qaoa, (-1,len_iter_qaoa))
    t_qaoa = np.reshape(t_qaoa, (-1,len_iter_qaoa))

    len_iter_vqe = np.max(nfev_vqe)//2+1
    p_vqe = np.reshape(p_vqe, (-1,len_iter_vqe))
    s_vqe = np.reshape(s_vqe, (-1,len_iter_vqe))
    nfev_vqe = np.reshape(nfev_vqe, (-1,len_iter_vqe))
    t_vqe = np.reshape(t_vqe, (-1,len_iter_vqe))

    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,6))
    for j in range(p_qaoa.shape[1]):
        ax1.plot(s_qaoa[:,j], p_qaoa[:,j], '--s', label=nfev_qaoa[0,j])
    for j in range(p_vqe.shape[1]):
        ax2.plot(s_vqe[:,j], p_vqe[:,j], '--s', label=nfev_vqe[0,j])

    ax1.set_xlabel('Shots')
    ax1.set_xscale('log', base=2)
    ax1.set_ylabel(r'$F(success \leq t)$')
    #ax1.set_ylim(0,1.1)

    ax2.set_xlabel('Shots')
    ax2.set_xscale('log', base=2)
    ax2.set_ylabel(r'$F(success \leq t)$')
    #ax2.set_ylim(0,1.1)

    ax1.legend(title='Iterations')
    ax2.legend(title='Iterations')

    ax1.set_title(f'QAOA+ N={4+k*2}')
    ax2.set_title(f'VQE N={4+k*2}')


### Comparison QAOA+ and VQE for $t$


In [ ]:
for k, (p_qaoa, s_qaoa, nfev_qaoa, t_qaoa, p_vqe, s_vqe, nfev_vqe, t_vqe) in enumerate(zip(psucc_qaoap, shots_qaoap, nfevs_qaoap, ts_qaoap, psucc_vqe, shots_vqe, nfevs_vqe, ts_vqe)):
    len_iter_qaoa = np.max(nfev_qaoa)//2+1 if 4+k*2 < 14 else np.max(nfev_qaoa)//5+1
    p_qaoa = np.reshape(p_qaoa, (-1,len_iter_qaoa))
    s_qaoa = np.reshape(s_qaoa, (-1,len_iter_qaoa))
    nfev_qaoa = np.reshape(nfev_qaoa, (-1,len_iter_qaoa))
    t_qaoa = np.reshape(t_qaoa, (-1,len_iter_qaoa))*2

    len_iter_vqe = np.max(nfev_vqe)//2+1
    p_vqe = np.reshape(p_vqe, (-1,len_iter_vqe))
    s_vqe = np.reshape(s_vqe, (-1,len_iter_vqe))
    nfev_vqe = np.reshape(nfev_vqe, (-1,len_iter_vqe))
    t_vqe = np.reshape(t_vqe, (-1,len_iter_vqe))*2

    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,6))
    for i in range(p_qaoa.shape[0]):
        ax1.plot(t_qaoa[i,:], p_qaoa[i,:], '--o', label=s_qaoa[i,0])
    ax1.plot(np.arange(0,2**(7+k*2),2), 1 - (1-1/(2**(4+k*2)))**np.arange(0,2**(7+k*2),2), '--', color="black", label="Random sampling")
    for i in range(p_vqe.shape[0]):
        ax2.plot(t_vqe[i,:], p_vqe[i,:], '--o', label=s_vqe[i,0])
    ax2.plot(np.arange(0,2**(7+k*2),2), 1 - (1-1/(2**(4+k*2)))**np.arange(0,2**(7+k*2),2), '--', color="black", label="Random sampling")
    #ax1.plot(np.arange(0, 2**(2*(4+k*2))), 1 - (1-1/(2**(4+k*2)))**np.arange(0, 2**(2*(4+k*2))), '--', color="black", label="Random sampling")

    ax1.set_xlabel('t (shots x iterations)')
    ax1.set_ylabel(r'$F(success \leq t)$')
    ax1.set_xscale('log', base=2)
    #ax1.set_yscale('log', base=2)
    #ax1.set_ylim(0,1.1)

    ax2.set_xlabel('t (shots x iterations)')
    ax2.set_ylabel(r'$F(success \leq t)$')
    ax2.set_xscale('log', base=2)
    #ax2.set_yscale('log', base=2)
    #ax2.set_ylim(0,1.1)

    ax1.legend(title='Shots')
    ax2.legend(title='Shots')

    ax1.set_title(f'QAOA+ N={4+k*2}')
    ax2.set_title(f'VQE N={4+k*2}')

    # if 4+k*2 == 14:
    #     plt.savefig("qaoa_vqe.png")


### QAOA+ $p=1$ vs VQE $p=1$ and QAOA+ $p=2$

Comparison only for size $N=12$.

In [ ]:
dir_path = f"results/N12/qaoa+_p1/"
print(f"directory: {dir_path}")
p_qaoa = []
t_qaoa = []
s_qaoa = []
nfev_qaoa = []
# for each number of qubits
# we have several number of shots and iterations
for filename in sorted(os.listdir(dir_path)):
    filename = dir_path + filename
    with open(filename, "r") as file:
        # print(filename)
        data = json.load(file)
    ever_found = []
    # for each shot and iteration param
    # we randomized the initial point
    # to estimate the right probability
    for run in data:
        ever_found.append(run["ever_found"])
    # compute p('found minimum')
    p_qaoa.append(
        np.mean(np.asarray(ever_found, dtype=np.float128))
    )  # float128 to avoid to many zeros
    # maxiter*shots = actual number of iteration
    t_qaoa.append(run["shots"] * run["nfev"])
    s_qaoa.append(run["shots"])
    nfev_qaoa.append(run["nfev"])

In [ ]:
k = 4
p_vqe, s_vqe, nfev_vqe, t_vqe = psucc_1[k], shots_1[k], nfevs_1[k], ts_1[k]


len_iter_qaoa = np.max(nfev_qaoa)//2+1 if 4+k*2 < 14 else np.max(nfev_qaoa)//5+1
p_qaoa = np.reshape(p_qaoa, (-1,len_iter_qaoa))
s_qaoa = np.reshape(s_qaoa, (-1,len_iter_qaoa))
nfev_qaoa = np.reshape(nfev_qaoa, (-1,len_iter_qaoa))
t_qaoa = np.reshape(t_qaoa, (-1,len_iter_qaoa))

len_iter_vqe = np.max(nfev_vqe)//2+1
p_vqe = np.reshape(p_vqe, (-1,len_iter_vqe))
s_vqe = np.reshape(s_vqe, (-1,len_iter_vqe))
nfev_vqe = np.reshape(nfev_vqe, (-1,len_iter_vqe))
t_vqe = np.reshape(t_vqe, (-1,len_iter_vqe))

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,6))
for i in range(p_qaoa.shape[0]):
    ax1.plot(t_qaoa[i,:], p_qaoa[i,:], '--o', label=s_qaoa[i,0])
ax1.plot(np.arange(0,2**(7+k*2),2), 1 - (1-1/(2**(4+k*2)))**np.arange(0,2**(7+k*2),2), '--', color="black", label="Random sampling")
for i in range(p_vqe.shape[0]):
    ax2.plot(t_vqe[i,:], p_vqe[i,:], '--o', label=s_vqe[i,0])
ax2.plot(np.arange(0,2**(7+k*2),2), 1 - (1-1/(2**(4+k*2)))**np.arange(0,2**(7+k*2),2), '--', color="black", label="Random sampling")
#ax1.plot(np.arange(0, 2**(2*(4+k*2))), 1 - (1-1/(2**(4+k*2)))**np.arange(0, 2**(2*(4+k*2))), '--', color="black", label="Random sampling")

ax1.set_xlabel('t (shots x iterations)')
ax1.set_ylabel(r'$F(success \leq t)$')
ax1.set_xscale('log', base=2)
#ax1.set_yscale('log', base=2)
#ax1.set_ylim(0,1.1)

ax2.set_xlabel('t (shots x iterations)')
ax2.set_ylabel(r'$F(success \leq t)$')
ax2.set_xscale('log', base=2)
#ax2.set_yscale('log', base=2)
#ax2.set_ylim(0,1.1)

ax1.legend(title='Shots')
ax2.legend(title='Shots')

ax1.set_title(f'QAOA+ N={4+k*2}')
ax2.set_title(f'VQE N={4+k*2}')

In [ ]:
k=4
p_qaoa_p2, s_qaoa_p2, nfev_qaoa_p2, t_qaoa_p2 = psucc_qaoap[k], shots_qaoap[k], nfevs_qaoap[k], ts_qaoap[k]


len_iter_qaoa = np.max(nfev_qaoa)//2+1 if 4+k*2 < 14 else np.max(nfev_qaoa)//5+1
p_qaoa = np.reshape(p_qaoa, (-1,len_iter_qaoa))
s_qaoa = np.reshape(s_qaoa, (-1,len_iter_qaoa))
nfev_qaoa = np.reshape(nfev_qaoa, (-1,len_iter_qaoa))
t_qaoa = np.reshape(t_qaoa, (-1,len_iter_qaoa))

len_iter_vqe = np.max(nfev_qaoa_p2)//2+1
p_qaoa_p2 = np.reshape(p_qaoa_p2, (-1,len_iter_vqe))
s_qaoa_p2 = np.reshape(s_qaoa_p2, (-1,len_iter_vqe))
nfev_qaoa_p2 = np.reshape(nfev_qaoa_p2, (-1,len_iter_vqe))
t_qaoa_p2 = np.reshape(t_qaoa_p2, (-1,len_iter_vqe))

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,6))
for i in range(p_qaoa.shape[0]):
    ax1.plot(t_qaoa[i,:], p_qaoa[i,:], '--o', label=s_qaoa[i,0])
ax1.plot(np.arange(0,2**(7+k*2),2), 1 - (1-1/(2**(4+k*2)))**np.arange(0,2**(7+k*2),2), '--', color="black", label="Random sampling")
for i in range(p_qaoa_p2.shape[0]):
    ax2.plot(t_qaoa_p2[i,:]*2, p_qaoa_p2[i,:], '--o', label=s_qaoa_p2[i,0])
ax2.plot(np.arange(0,2**(7+k*2),2), 1 - (1-1/(2**(4+k*2)))**np.arange(0,2**(7+k*2),2), '--', color="black", label="Random sampling")
#ax1.plot(np.arange(0, 2**(2*(4+k*2))), 1 - (1-1/(2**(4+k*2)))**np.arange(0, 2**(2*(4+k*2))), '--', color="black", label="Random sampling")

ax1.set_xlabel('t (shots x iterations)')
ax1.set_ylabel(r'$F(success \leq t)$')
ax1.set_xscale('log', base=2)
#ax1.set_yscale('log', base=2)
#ax1.set_ylim(0,1.1)

ax2.set_xlabel('t (shots x iterations)')
ax2.set_ylabel(r'$F(success \leq t)$')
ax2.set_xscale('log', base=2)
#ax2.set_yscale('log', base=2)
#ax2.set_ylim(0,1.1)

ax1.legend(title='Shots')
ax2.legend(title='Shots')

ax1.set_title(f'QAOA+ p=1 N={4+k*2}')
ax2.set_title(f'QAOA+ p=2 N={4+k*2}')


## QAOA $p=2$ vs VQE


In [ ]:
import numpy as np
from scipy.optimize import curve_fit
from math import ceil
from json import JSONEncoder
from matplotlib import pyplot as plt
from src.utils import collect_results, NumpyArrayEncoder, create_ansatz
import json
import os

In [ ]:
qubits = [8,10,12,14,16,18]
p = 2
# load exact simulations
psucc_vqe, ts_vqe, shots_vqe, nfevs_vqe = collect_results(qubits, p)

In [ ]:
ts_qaoa = []
shots_qaoa = []
nfevs_qaoa = []
psucc_qaoa = []
for qubit in qubits:
    dir_path = f"results/N{qubit}/qaoa_p2/" 
    print(f"directory: {dir_path}")
    p = []
    t = []
    s = []
    it = []
    # for each number of qubits
    # we have several number of shots and iterations
    for filename in sorted(os.listdir(dir_path)):
        filename = dir_path + filename
        with open(filename, "r") as file:
            # print(filename)
            data = json.load(file)
        ever_found = []
        # for each shot and iteration param
        # we randomized the initial point
        # to estimate the right probability
        for run in data:
            ever_found.append(run["ever_found"])
        # compute p('found minimum')
        p.append(
            np.mean(np.asarray(ever_found, dtype=np.float128))
        )  # float128 to avoid to many zeros
        # maxiter*shots = actual number of iteration
        t.append(run["shots"] * run["nfev"])
        s.append(run["shots"])
        it.append(run["nfev"])
    # update list for each number of qubits
    ts_qaoa.append(t)
    psucc_qaoa.append(p)
    shots_qaoa.append(s)
    nfevs_qaoa.append(it)

In [ ]:
colors = ['lightgreen','forestgreen','tab:green','darkgreen', 'lightseagreen', 'tab:cyan','cornflowerblue' , 'blue', 'navy']
plots1 = []
plots2 = []

for i, (p,t) in enumerate(zip(psucc_vqe, ts_vqe)):
    if 4+i*2 > 18:
        continue
    try:
        last_zero = np.argwhere(np.isclose(p, 0.))
        print(f"N={4+i*2}, p=2, t={np.asarray(t)[np.asarray(p)==0.].max()}, shots={np.asarray(shots_vqe[i])[np.asarray(p)==0.].max()}, nfev={np.asarray(nfevs_vqe[i])[np.asarray(p)==0.].max()}")
    except ValueError:
        print(f"No zeros in success probability for N={4+i*2}")
    # plot for each dimension
    p_t = np.vstack((t, p))
    p_t = p_t[:, np.argsort(p_t[0, :])]
    plot1, = plt.plot(p_t[0]*2, p_t[1], ".", label=f"{4+i*2}", color=colors[i], alpha=0.8)
    plots1.append(plot1)

for i, (p,t) in enumerate(zip(psucc_qaoa, ts_qaoa)):
    try:
        last_zero = np.argwhere(np.isclose(psucc_qaoa[i], 0.))
        print(f"N={4+i*2}, p=2, t={np.asarray(ts_qaoa[i])[np.asarray(psucc_qaoa[i])==0.].max()}, shots={np.asarray(shots_qaoa[i])[np.asarray(psucc_qaoa[i])==0.].max()}, nfev={np.asarray(nfevs_qaoa[i])[np.asarray(psucc_qaoa[i])==0.].max()}")
    except ValueError:
        print(f"No zeros in succes probability for N={4+i*2}")
    p_t_qaoa = np.vstack((ts_qaoa[i], psucc_qaoa[i]))
    p_t_qaoa = p_t_qaoa[:, np.argsort(p_t_qaoa[0, :])]
    plot2, = plt.plot(p_t_qaoa[0]*2, p_t_qaoa[1], ".", label=f"{4+i*2}", color=colors[i], fillstyle='none')
    plots2.append(plot2)

    plt.xscale('log')
    plt.xlabel(r"$t$ (shots x iterations x depth)")
    plt.ylabel(r"$P$")

# Create a legend for the first line.
first_legend = plt.legend(handles=plots1, loc='lower right', title="VQE")
#Add the legend manually to the current Axes.
ax = plt.gca().add_artist(first_legend)

plt.title(r"$p=2$")
#Create another legend for the second line.
plt.legend(handles=plots2, loc='upper left', title=f"QAOA")

In [ ]:
def fun_exp(x, a, b):
    return a*2**(b*x)

def fun_lin(x, a, b):
    return a + b*x
    
pt_all = []
for i, (p,t) in enumerate(zip(psucc_vqe, ts_vqe)):
    p_t = np.vstack((t, p))
    p_t[0] *= 2 
    p_t = p_t[:, np.argsort(p_t[0, :])]
    pt_all.append(p_t)
pt_all_2 = []
for i, (p,t) in enumerate(zip(psucc_qaoa, ts_qaoa)):
    p_t = np.vstack((t, p))
    p_t[0] *= 2 
    p_t = p_t[:, np.argsort(p_t[0, :])]
    pt_all_2.append(p_t)


colors = ['lightgreen', 'limegreen', 'forestgreen', 'darkgreen','lightseagreen', 'cornflowerblue' , 'blue', 'navy']
min_psucc_lst = [0.99, 0.9, 0.5, 0.25]
min_qubits = 8 
max_qubits = 18
plots1 = [] 
plots2 = []

plt.plot(np.arange(min_qubits, max_qubits+1), fun_exp(np.arange(min_qubits, max_qubits+1), 1, 1), color='red', linewidth=3.5, linestyle="dashed")

for min_psucc, color in zip(min_psucc_lst, colors[::2]):
    step_size = []
    for i, pt in enumerate(pt_all):
        mask = pt[1] >= min_psucc
        # check if the p is not reached
        if mask.sum() == 0:
            continue
        step_size.append(pt[0][mask][0])

    x_axes = 8 + 2*np.arange(len(step_size))
    popt, _ = curve_fit(fun_lin, x_axes, np.log2(step_size))
    plot1, = plt.plot(x_axes, step_size, "o", label=f"p={min_psucc}", color=color)
    plots1.append(plot1)
    plot1, = plt.plot(np.arange(min_qubits, max_qubits+1), fun_exp(np.arange(min_qubits, max_qubits+1), 2**popt[0], popt[1]), "-", label=f"a={2**popt[0]:.2f}, b={popt[1]:.2f}", color=color)
    plots1.append(plot1)

    step_size_2 = []
    for i, pt_2 in enumerate(pt_all_2):
        mask = pt_2[1] >= min_psucc
        # check if the p is not reached
        if mask.sum() == 0:
            continue
        step_size_2.append(pt_2[0][mask][0])

    x_axes_2 = 8 + 2*np.arange(len(step_size_2))
    popt, _ = curve_fit(fun_lin, x_axes_2, np.log2(step_size_2))
    plot2, = plt.plot(x_axes_2, step_size_2, "o", label=f"p={min_psucc}", color=color, fillstyle='none')
    plots2.append(plot2)
    plot2, = plt.plot(np.arange(min_qubits, max_qubits+1), fun_exp(np.arange(min_qubits, max_qubits+1), 2**popt[0], popt[1]), "--", label=f"a={2**popt[0]:.2f}, b={popt[1]:.2f}", color=color)
    plots2.append(plot2)

plt.yscale('log', base=2)
plt.ylabel(r"$t$ (shots x iterations)")
plt.xlabel(r"$N$")


# Create a legend for the first line.
first_legend = plt.legend(handles=plots1, loc='upper left', title="VQE", fontsize="x-small")
#Add the legend manually to the current Axes.
ax = plt.gca().add_artist(first_legend)

plt.title(r"$p=2$")
#Create another legend for the second line.
plt.legend(handles=plots2, loc='upper right', title="QAOA", fontsize="x-small")

plt.title(r"Fit with $a \cdot 2^{b \cdot N}$")

plt.savefig("scale_qaoap2.png")

In [ ]:
t_108 = fun_exp(108, 166, 0.39) / 2
print(f"{t_108:.2e} circuit repetions (shots x iterations)")
print(f"{t_108*10e-9:.2e}s")

In [ ]:
for k, (p_qaoa, s_qaoa, nfev_qaoa, t_qaoa, p_vqe, s_vqe, nfev_vqe, t_vqe) in enumerate(zip(psucc_qaoa, shots_qaoa, nfevs_qaoa, ts_qaoa, psucc_vqe, shots_vqe, nfevs_vqe, ts_vqe)):
    len_iter_qaoa = np.max(nfev_qaoa)//2+1
    #print(len_iter_qaoa, np.max(nfev_qaoa), len(s_qaoa), 4+k*2)
    p_qaoa = np.reshape(p_qaoa, (-1,len_iter_qaoa))
    s_qaoa = np.reshape(s_qaoa, (-1,len_iter_qaoa))
    nfev_qaoa = np.reshape(nfev_qaoa, (-1,len_iter_qaoa))
    t_qaoa = np.reshape(t_qaoa, (-1,len_iter_qaoa))*2

    len_iter_vqe = np.max(nfev_vqe)//2+1
    p_vqe = np.reshape(p_vqe, (-1,len_iter_vqe))
    s_vqe = np.reshape(s_vqe, (-1,len_iter_vqe))
    nfev_vqe = np.reshape(nfev_vqe, (-1,len_iter_vqe))
    t_vqe = np.reshape(t_vqe, (-1,len_iter_vqe))*2

    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,6))
    for i in range(p_qaoa.shape[0]):
        ax1.plot(t_qaoa[i,:], p_qaoa[i,:], '--o', label=s_qaoa[i,0])
    ax1.plot(np.arange(0, 2**(8+k*2)), 1 - (1-1/(2**(4+k*2)))**np.arange(0, 2**(8+k*2)), '--', color="black", label="Random sampling")
    for i in range(p_vqe.shape[0]):
        ax2.plot(t_vqe[i,:], p_vqe[i,:], '--o', label=s_vqe[i,0])
    ax2.plot(np.arange(0, 2**(8+k*2)), 1 - (1-1/(2**(4+k*2)))**np.arange(0, 2**(8+k*2)), '--', color="black", label="Random sampling")

    ax1.set_xlabel('t (shots x iterations)')
    ax1.set_ylabel(r'$F(success \leq t)$')
    ax1.set_xscale('log', base=2)
    #ax1.set_yscale('log', base=2)
    #ax1.set_ylim(0,1.1)

    ax2.set_xlabel('t')
    ax2.set_ylabel(r'$F(success \leq t)$')
    ax2.set_xscale('log', base=2)
    #ax2.set_yscale('log', base=2)
    #ax2.set_ylim(0,1.1)

    ax1.legend(title='Shots')
    ax2.legend(title='Shots')

    ax1.set_title(f'QAOA N={4+k*2}')
    ax2.set_title(f'VQE N={4+k*2}')

    #if 4+k*2 == 14:
    #    plt.savefig("qaoa_vqe.png")


## QAOA $p=3$ vs QAOA $p=2$

Comparison between QAOA (classic version) with different depth.

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
from math import ceil
from json import JSONEncoder
from matplotlib import pyplot as plt
from src.utils import collect_results, NumpyArrayEncoder, create_ansatz
import json
import os

In [ ]:
qubits = [4,6,8,10,12,14,16,18]
p = 2

In [ ]:
ts_qaoa2 = []
shots_qaoa2 = []
nfevs_qaoa2 = []
psucc_qaoa2 = []
for qubit in qubits:
    dir_path = f"results/N{qubit}/qaoa_p2/"
    print(f"directory: {dir_path}")
    p = []
    t = []
    s = []
    it = []
    # for each number of qubits
    # we have several number of shots and iterations
    for filename in sorted(os.listdir(dir_path)):
        filename = dir_path + filename
        with open(filename, "r") as file:
            # print(filename)
            data = json.load(file)
        ever_found = []
        # for each shot and iteration param
        # we randomized the initial point
        # to estimate the right probability
        for run in data:
            ever_found.append(run["ever_found"])
        # compute p('found minimum')
        p.append(
            np.mean(np.asarray(ever_found, dtype=np.float128))
        )  # float128 to avoid to many zeros
        # maxiter*shots = actual number of iteration
        t.append(run["shots"] * run["nfev"])
        s.append(run["shots"])
        it.append(run["nfev"])
    # update list for each number of qubits
    ts_qaoa2.append(t)
    psucc_qaoa2.append(p)
    shots_qaoa2.append(s)
    nfevs_qaoa2.append(it)

In [ ]:
ts_qaoa3 = []
shots_qaoa3 = []
nfevs_qaoa3 = []
psucc_qaoa3 = []
for qubit in qubits:
    dir_path = f"results/N{qubit}/qaoa_p3/"
    print(f"directory: {dir_path}")
    p = []
    t = []
    s = []
    it = []
    # for each number of qubits
    # we have several number of shots and iterations
    for filename in sorted(os.listdir(dir_path)):
        filename = dir_path + filename
        with open(filename, "r") as file:
            # print(filename)
            data = json.load(file)
        ever_found = []
        # for each shot and iteration param
        # we randomized the initial point
        # to estimate the right probability
        for run in data:
            ever_found.append(run["ever_found"])
        # compute p('found minimum')
        p.append(
            np.mean(np.asarray(ever_found, dtype=np.float128))
        )  # float128 to avoid to many zeros
        # maxiter*shots = actual number of iteration
        t.append(run["shots"] * run["nfev"])
        s.append(run["shots"])
        it.append(run["nfev"])
    # update list for each number of qubits
    ts_qaoa3.append(t)
    psucc_qaoa3.append(p)
    shots_qaoa3.append(s)
    nfevs_qaoa3.append(it)

In [ ]:
colors = ['lightgreen','forestgreen','tab:green','darkgreen', 'lightseagreen', 'tab:cyan','cornflowerblue' , 'blue', 'navy']
plots1 = []
plots2 = []

for i, (p,t) in enumerate(zip(psucc_qaoa2, ts_qaoa2)):
    if 4+i*2 > 18:
        continue
    try:
        last_zero = np.argwhere(np.isclose(p, 0.))
        print(f"N={4+i*2}, p=2, t={np.asarray(t)[np.asarray(p)==0.].max()}, shots={np.asarray(shots_qaoa2[i])[np.asarray(p)==0.].max()}, nfev={np.asarray(nfevs_qaoa2[i])[np.asarray(p)==0.].max()}")
    except ValueError:
        print(f"No zeros in success probability for N={4+i*2}")
    # plot for each dimension
    p_t = np.vstack((t, p))
    p_t = p_t[:, np.argsort(p_t[0, :])]
    plot1, = plt.plot(p_t[0]*2, p_t[1], ".", label=f"{4+i*2}", color=colors[i], alpha=0.8)
    plots1.append(plot1)

for i, (p,t) in enumerate(zip(psucc_qaoa3, ts_qaoa3)):
    try:
        last_zero = np.argwhere(np.isclose(psucc_qaoa3[i], 0.))
        print(f"N={4+i*2}, p=2, t={np.asarray(ts_qaoa3[i])[np.asarray(psucc_qaoa3[i])==0.].max()}, shots={np.asarray(shots_qaoa3[i])[np.asarray(psucc_qaoa3[i])==0.].max()}, nfev={np.asarray(nfevs_qaoa3[i])[np.asarray(psucc_qaoa3[i])==0.].max()}")
    except ValueError:
        print(f"No zeros in succes probability for N={4+i*2}")
    p_t_qaoa3 = np.vstack((ts_qaoa3[i], psucc_qaoa3[i]))
    p_t_qaoa3 = p_t_qaoa3[:, np.argsort(p_t_qaoa3[0, :])]
    plot2, = plt.plot(p_t_qaoa3[0]*2, p_t_qaoa3[1], ".", label=f"{4+i*2}", color=colors[i], fillstyle='none')
    plots2.append(plot2)

    plt.xscale('log')
    plt.xlabel(r"$t$ (shots x iterations x depth)")
    plt.ylabel(r"$P$")

# Create a legend for the first line.
first_legend = plt.legend(handles=plots1, loc='lower right', title=r"QAOA $p=2$")
#Add the legend manually to the current Axes.
ax = plt.gca().add_artist(first_legend)

plt.title(r"$p=2$")
#Create another legend for the second line.
plt.legend(handles=plots2, loc='upper left', title=r"QAOA $p=3$")


### Clean results

Clean repeated $t$ and consider only the best one, i.e., the one with the high success porbability.

In [ ]:
new_psucc_qaoa2 = []
new_ts_qaoa2 = []
for i, (ps,ts) in enumerate(zip(psucc_qaoa2, ts_qaoa2)):
    # plot for each dimension
    p_t = np.vstack((ts, ps))
    p_t = p_t[:, np.argsort(p_t[0, :])]
    new_p = [p_t[1,0]]
    new_t = [p_t[0,0]]
    new_len = 0
    for j, (t,p) in enumerate(p_t.T):
        if new_t[new_len] == t:
            if new_p[new_len] > p:
                continue
            new_p[new_len] = p
        else:
            new_t.append(t)
            new_p.append(p)
            new_len += 1
    new_psucc_qaoa2.append(new_p)
    new_ts_qaoa2.append(new_t)

In [ ]:
def fun_exp(x, a, b):
    return a*2**(b*x)

def fun_lin(x, a, b):
    return a + b*x
    
pt_all = []
for i, (p,t) in enumerate(zip(psucc_qaoa2, ts_qaoa2)):
    p_t = np.vstack((t, p))
    p_t[0] *= 2 
    p_t = p_t[:, np.argsort(p_t[0, :])]
    pt_all.append(p_t)
pt_all_2 = []
for i, (p,t) in enumerate(zip(psucc_qaoa3, ts_qaoa3)):
    p_t = np.vstack((t, p))
    p_t[0] *= 3 
    p_t = p_t[:, np.argsort(p_t[0, :])]
    pt_all_2.append(p_t)


colors = ['lightgreen', 'limegreen', 'forestgreen', 'darkgreen','lightseagreen', 'cornflowerblue' , 'blue', 'navy']
min_psucc_lst = [0.99, 0.9, 0.5, 0.25]
min_qubits = 4 
max_qubits = 18
plots1 = [] 
plots2 = []

plt.plot(np.arange(min_qubits, max_qubits+1), fun_exp(np.arange(min_qubits, max_qubits+1), 1, 1), color='red', linewidth=3.5, linestyle="dashed")

for min_psucc, color in zip(min_psucc_lst, colors[::2]):
    step_size = []
    for i, pt in enumerate(pt_all):
        mask = pt[1] >= min_psucc
        # check if the p is not reached
        if mask.sum() == 0:
            continue
        step_size.append(pt[0][mask][0])

    x_axes = 4 + 2*np.arange(len(step_size))

    popt, _ = curve_fit(fun_lin, x_axes, np.log2(step_size))
    plot1, = plt.plot(x_axes, step_size, "o", label=f"p={min_psucc}", color=color)
    plots1.append(plot1)
    plot1, = plt.plot(np.arange(min_qubits, max_qubits+1), fun_exp(np.arange(min_qubits, max_qubits+1), 2**popt[0], popt[1]), "-", label=f"a={2**popt[0]:.2f}, b={popt[1]:.2f}", color=color)
    plots1.append(plot1)

    step_size_2 = []
    for i, pt_2 in enumerate(pt_all_2):
        mask = pt_2[1] >= min_psucc
        # check if the p is not reached
        if mask.sum() == 0:
            continue
        step_size_2.append(pt_2[0][mask][0])
    x_axes_2 = 4 + 2*np.arange(len(step_size_2))
    popt, _ = curve_fit(fun_lin, x_axes_2, np.log2(step_size_2))
    plot2, = plt.plot(x_axes_2, step_size_2, "o", label=f"p={min_psucc}", color=color, fillstyle='none')
    plots2.append(plot2)
    plot2, = plt.plot(np.arange(min_qubits, max_qubits+1), fun_exp(np.arange(min_qubits, max_qubits+1), 2**popt[0], popt[1]), "--", label=f"a={2**popt[0]:.2f}, b={popt[1]:.2f}", color=color)
    plots2.append(plot2)

plt.yscale('log', base=2)
plt.ylabel(r"$t$ (shots x iterations)")
plt.xlabel(r"$N$")


# Create a legend for the first line.
first_legend = plt.legend(handles=plots1, loc='upper left', title=r"QAOA $p=2$", fontsize="x-small")
#Add the legend manually to the current Axes.
ax = plt.gca().add_artist(first_legend)

plt.title(r"$p=2$")
#Create another legend for the second line.
plt.legend(handles=plots2, loc='lower right', title=r"QAOA $p=3$", fontsize="x-small")

plt.title(r"Fit with $a \cdot 2^{b \cdot N}$")

#plt.savefig("scale_qaoap2.png")

In [ ]:
for k, (p_qaoa2, s_qaoa2, nfev_qaoa2, t_qaoa2, p_qaoa3, s_qaoa3, nfev_qaoa3, t_qaoa3) in enumerate(zip(psucc_qaoa2, shots_qaoa2, nfevs_qaoa2, ts_qaoa2, psucc_qaoa3, shots_qaoa3, nfevs_qaoa3, ts_qaoa3)):
    len_iter_qaoa2 = np.max(nfev_qaoa2)//2+1
    p_qaoa2 = np.reshape(p_qaoa2, (-1,len_iter_qaoa2))
    s_qaoa2 = np.reshape(s_qaoa2, (-1,len_iter_qaoa2))
    nfev_qaoa2 = np.reshape(nfev_qaoa2, (-1,len_iter_qaoa2))
    t_qaoa2 = np.reshape(t_qaoa2, (-1,len_iter_qaoa2)) * 2

    len_iter_qaoa3 = np.max(nfev_qaoa3)//2+1
    p_qaoa3 = np.reshape(p_qaoa3, (-1,len_iter_qaoa3))
    s_qaoa3 = np.reshape(s_qaoa3, (-1,len_iter_qaoa3))
    nfev_qaoa3 = np.reshape(nfev_qaoa3, (-1,len_iter_qaoa3))
    t_qaoa3 = np.reshape(t_qaoa3, (-1,len_iter_qaoa3)) * 3

    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,6))
    for i in range(p_qaoa2.shape[0]):
        ax1.plot(t_qaoa2[i,:], p_qaoa2[i,:], '--o', label=s_qaoa2[i,0])
    ax1.plot(np.arange(0, 2**(8+k*2)), 1 - (1-1/(2**(4+k*2)))**np.arange(0, 2**(8+k*2)), '--', color="black", label="Random sampling")
    for i in range(p_qaoa3.shape[0]):
        ax2.plot(t_qaoa3[i,:], p_qaoa3[i,:], '--o', label=s_qaoa3[i,0])
    ax2.plot(np.arange(0, 2**(8+k*2)), 1 - (1-1/(2**(4+k*2)))**np.arange(0, 2**(8+k*2)), '--', color="black", label="Random sampling")

    ax1.set_xlabel('t (shots x iterations)')
    ax1.set_ylabel(r'$F(success \leq t)$')
    ax1.set_xscale('log', base=2)
    #ax1.set_yscale('log', base=2)
    #ax1.set_ylim(0,1.1)

    ax2.set_xlabel('t')
    ax2.set_ylabel(r'$F(success \leq t)$')
    ax2.set_xscale('log', base=2)
    #ax2.set_yscale('log', base=2)
    #ax2.set_ylim(0,1.1)

    ax1.legend(title='Shots')
    ax2.legend(title='Shots')

    ax1.set_title(f'QAOA p=2 N={4+k*2}')
    ax2.set_title(f'QAOA p=3 N={4+k*2}')


## Test circuits with noise

Implement RealAmplitude ansatz and look for reliable noise setting. I load the noise from a qiskit class, from the actual real divece and I compare them with the exact simulation.

In [ ]:
from math import pi
import numpy as np
import matplotlib.pyplot as plt

from qiskit.circuit.library import  RealAmplitudes
from qiskit.providers.fake_provider import FakeMumbaiV2, FakeVigoV2, FakeMumbai 
from qiskit.providers.aer import AerProvider, AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit import IBMQ, transpile
from qiskit.visualization import plot_histogram
from qiskit.providers.aer.noise import thermal_relaxation_error, depolarizing_error


### Fake Device

Loads from qiskit class a shot of a real device.

In [ ]:
# Mumbai has 27 qubots and quantum volume 128
device_backend = FakeMumbai()
noise_model = NoiseModel.from_backend(device_backend)
sim_fakenoise = AerSimulator(
            method='statevector',
            max_parallel_threads=1,
            noise_model = noise_model, 
        )


### Custom noise

Implement custom noise in two different ways, setting damping time $T_1$ and dephasing time $T_2$ or depolarizing error $p_1$.

In [ ]:
# T1 and T2 values for all qubits
# T1s = np.random.normal(50e3, 10e3) # Sampled from normal distribution mean 50 microsec
# T2s = np.random.normal(70e3, 10e3)  # Sampled from normal distribution mean 50 microsec

t1 = 50e3
t2 = 70e3

# Truncate random T2s <= T1s
#T2s = np.array([min(T2s[j], 2 * T1s[j]) for j in range(20)])

# Instruction times (in nanoseconds)
time_u1 = 0   # virtual gate
time_u2 = 50  # (single X90 pulse)
time_u3 = 100 # (two X90 pulses)
time_cx = 300
time_reset = 1000  # 1 microsecond
time_measure = 1000 # 1 microsecond


# QuantumError objects
errors_reset = thermal_relaxation_error(t1, t2, time_reset)
errors_measure = thermal_relaxation_error(t1, t2, time_measure)
errors_u1  = thermal_relaxation_error(t1, t2, time_u1)
errors_u2  = thermal_relaxation_error(t1, t2, time_u2)
errors_u3  = thermal_relaxation_error(t1, t2, time_u3)
errors_cx = thermal_relaxation_error(t1, t2, time_cx).expand(thermal_relaxation_error(t1, t2, time_cx))
# Add errors to noise model
noise_thermal = NoiseModel()
noise_thermal.add_all_qubit_quantum_error(errors_reset, "reset")
noise_thermal.add_all_qubit_quantum_error(errors_measure, "measure")
noise_thermal.add_all_qubit_quantum_error(errors_u1, "u1")
noise_thermal.add_all_qubit_quantum_error(errors_u2, "u2")
noise_thermal.add_all_qubit_quantum_error(errors_u3, "u3")

noise_thermal.add_all_qubit_quantum_error(errors_cx, "cx")


sim_customnoise = AerSimulator(
            method='statevector',
            max_parallel_threads=1,
            noise_model=noise_thermal, 
        )

In [ ]:
# depolarizing error

prob_1 = 1e-2 / 10. #1e-3 -> 1e-4 (best scenario)
prob_2 = prob_1 * 10. ##1e-2 -> 1e-3 (best) 
error_1 = depolarizing_error(prob_1, 1)
error_2 = depolarizing_error(prob_2, 2)

# Add errors to noise model
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(error_1, ['u1', 'u2', 'u3', 'h'])
noise_model.add_all_qubit_quantum_error(error_2, ['cx'])

sim_customnoise2 = AerSimulator(
            method='statevector',
            max_parallel_threads=1,
            noise_model=noise_model, 
        )


### Real device noise 

Load the noise on the fly from current online devices, the maximum qubits number is thus limited to 7. 

In [ ]:
# get a real backend from a real provider
provider = IBMQ.load_account()
# define noise from online ibm computer
backend = provider.get_backend('ibmq_manila')
noise_model = NoiseModel.from_backend(backend)
# generate a simulator that mimics the real quantum system with the latest calibration results
sim_realnoise = AerSimulator(
            method='statevector',
            max_parallel_threads=1,
            noise_model = noise_model, 
        )


### Exact simulation

We compare the noisy simulator with the exact one.

In [ ]:
exact_sim = AerSimulator(
            method='statevector',
            max_parallel_threads=1,
        )

In [ ]:
qubits = 4
circ_depth = 2
seed = 42


# create the circuit
# standard VQE ansatz
qc = RealAmplitudes(
    qubits,
    reps=circ_depth,
    insert_barriers=True,
    entanglement="circular",
)
# measure all qubits at the end of the circuit
qc.measure_all()

# define generator for initial point
rng = np.random.default_rng(seed=seed)
params =  np.ones(qc.num_parameters) * pi 
#params = np.random.uniform(-2 * pi, 2 * pi, qc.num_parameters)

qc = qc.assign_parameters(params)

print(qc)

In [ ]:
# Execute the circuit with fixed params
job = exact_sim.run(transpile(qc, exact_sim))
# Grab results from the job
result = job.result().get_counts()
# return counts
plot_histogram(result, title='Exact') #filename='exact_sim.png')

In [ ]:
# Execute the circuit with fixed params
job = sim_realnoise.run(transpile(qc, sim_realnoise))
# Grab results from the job
result = job.result().get_counts()
# return counts
plot_histogram(result, title='Real device Manila')#filename='realnoise.png')

In [ ]:
# Execute the circuit with fixed params
job = sim_fakenoise.run(transpile(qc, sim_fakenoise))
# Grab results from the job
result = job.result().get_counts()
# return counts
plot_histogram(result, title='Fake device Mumbai') #filename='fakenoise.png')

In [ ]:
# Execute the circuit with fixed params
job = sim_customnoise2.run(transpile(qc, sim_customnoise2))
# Grab results from the job
result = job.result().get_counts()
# return counts
plot_histogram(result, title=f'Depolarizing noise {prob_1*1e2}%') #filename='customnoise2.png')

In [ ]:
# Execute the circuit with fixed params
job = sim_customnoise.run(transpile(qc, sim_customnoise))
# Grab results from the job
result = job.result().get_counts()
# return counts
plot_histogram(result, title=f'Damping {t1//1e3}\u03bcs  Dephasing {t2//1e3}\u03bcs',)# filename='customnoise.png')


## Use Pandas

It's time to use Pandas.

In [ ]:
import itertools
import os
import math

import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt

In [ ]:
sizes = np.arange(4,22,2)
circ_depths = np.asarray([1,2,3,4])
algs = np.asarray(["qaoa", "qaoa+", "vqe"])

model_info = [(algo, p) for p in circ_depths for algo in algs] 

In [ ]:
run_info = {"t": [], "shots": [], "iter": [], "p_succ": [], "qubits": [], "circ_depth": [], "algo": [], "succ_true": [], "succ_false": [], 'p_succ_final': []}
for n in sizes:
    for (algo, p) in model_info:
        dir_path = f"results/N{n}/{algo}_p{p}/"
        if not os.path.isdir(dir_path):
            print(f"{dir_path} does not exist")
            continue
        for filename in sorted(os.listdir(dir_path)):
            filename = dir_path + filename
            with open(filename, "r") as file:
                # data are a list with #initial_points elements
                runs = json.load(file)
            ever_founds = []
            success_final = []
            iterations = [] 
            # for each shot and iteration param
            # we randomized the initial point
            # to estimate the right probability
            for r in runs:
                ever_founds.append(r["ever_found"])
                iterations.append(r["nfev"])
                success_final.append(r["success"])
            # compute p('found minimum')
            #run_info["p_succ"].append([ever_founds])
            run_info["p_succ"].append(
                np.mean(np.asarray(ever_founds, dtype=np.float128))
            )  # float128 to avoid to many zeros
            run_info["p_succ_final"].append(
                np.mean(np.asarray(success_final, dtype=np.float128))
            )  # float128 to avoid to many zeros
            run_info["succ_true"].append(np.count_nonzero(ever_founds))
            run_info["succ_false"].append(len(ever_founds) - np.count_nonzero(ever_founds))

            run_info["iter"].append(int(math.ceil(np.mean(np.asarray(iterations)))))
            # maxiter*shots = actual number of iteration
            run_info["t"].append(int(r["shots"] * run_info["iter"][-1]))
            run_info["shots"].append(int(r["shots"]))
            run_info["qubits"].append(int(n))
            run_info["circ_depth"].append(int(p))
            run_info["algo"].append(str(algo))
        
jobdf = pd.DataFrame.from_dict(run_info)
jobdf.to_csv(f"data_0pi_iter1.csv")

In [ ]:
jobdf.head(10)


### VQE with noise


In [ ]:
import itertools
import os
import math

import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt

In [ ]:
sizes = np.arange(4,14,2)
circ_depths = [2]
algs = ["vqe"]

model_info = [(n, algo, p) for p in circ_depths for algo in algs for n in sizes] 

In [ ]:
run_info = {"t": [], "shots": [], "iter": [], "p_succ": [], "qubits": [], "circ_depth": [], "algo": [], "succ_true": [], "succ_false": [], "noise": []}
for (n, algo, p) in model_info:
    dir_path = f"results/N{n}/noise/"
    if not os.path.isdir(dir_path):
        print(f"{dir_path} does not exist")
        continue
    for filename in sorted(os.listdir(dir_path)):
        filename = dir_path + filename
        with open(filename, "r") as file:
            # data are a list with #initial_points elements
            runs = json.load(file)
        ever_founds = []
        iterations = [] 
        # for each shot and iteration param
        # we randomized the initial point
        # to estimate the right probability
        for r in runs:
            ever_founds.append(r["ever_found"])
            iterations.append(r["nfev"])
        # compute p('found minimum')
        #run_info["p_succ"].append([ever_founds])
        run_info["p_succ"].append(
            np.mean(np.asarray(ever_founds, dtype=np.float128))
          )  # float128 to avoid to many zeros
        run_info["succ_true"].append(np.count_nonzero(ever_founds))
        run_info["succ_false"].append(len(ever_founds) - np.count_nonzero(ever_founds))

        run_info["iter"].append(int(math.ceil(np.mean(np.asarray(iterations)))))
        # maxiter*shots = actual number of iteration
        run_info["t"].append(int(r["shots"] * run_info["iter"][-1]))
        run_info["shots"].append(int(r["shots"]))
        run_info["qubits"].append(int(n))
        run_info["circ_depth"].append(int(p))
        run_info["algo"].append(str(algo))
        run_info["noise"].append(bool(True))
        
jobdf = pd.DataFrame.from_dict(run_info)
jobdf.to_csv(f"datanoise.csv")

In [ ]:
jobdf.head()

In [ ]:
for shot, data in jobdf[(jobdf['circ_depth'] == 2) & (jobdf['algo'] == 'qaoa')].groupby("shots"):
    print(f"{shot} {len(data['p_succ'])}", [p=='True' for p in data["p_succ"]])

In [ ]:
plt.plot(np.arange(0,2**(5+jobdf['qubits'][0]),2), 1 - (1-1/(2**jobdf['qubits'][0]))**np.arange(0,2**(5+jobdf['qubits'][0]),2), '--', color="black", label="Random sampling")
for shot, data in jobdf[(jobdf['circ_depth'] == 2) & (jobdf['algo'] == 'qaoa')].groupby("shots"):
    if shot > 64:
        continue
    #plt.plot(data["t"], data["p_succ"].apply(np.mean, 1, dtype=np.float128), "-o", label=f"{shot}")
    plt.plot(data["t"]*jobdf['circ_depth'].mean(), data["p_succ"], "-o", label=f"{shot}")
    plt.xscale("log", base=2)
plt.title(f"$N=8$ QAOA $p=2$")
plt.legend(title='Shots')

plt.savefig("qaoa_ferro.png")

In [ ]:
jobdf.to_csv("data_n6.csv")


## Comparison for Spin Glasses


In [ ]:
import itertools
import os
import math

import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt

In [ ]:
sizes = [16] 
circ_depths = [1,2,3,4]
algs = ["qaoa", "qaoa+", "vqe"]
seeds = np.arange(1, 31)

model_info = [(n, algo, p, seed) for p in circ_depths for algo in algs for seed in seeds for n in sizes]

In [ ]:
run_info = {"t": [], "shots": [], "iter": [], "p_succ": [], "qubits": [], "circ_depth": [], "algo": [], "seed": []}
for (n, algo, p, seed) in model_info:
    dir_path = f"results/N{n}/spinglass/{algo}p{p}_{seed}/"
    if not os.path.isdir(dir_path):
        #print(f"{dir_path} does not exist")
        continue
    for filename in sorted(os.listdir(dir_path)):
        filename = dir_path + filename
        with open(filename, "r") as file:
            # data are a list with #initial_points elements
            runs = json.load(file)
        ever_founds = []
        iterations = [] 
        # for each shot and iteration param
        # we randomized the initial point
        # to estimate the right probability
        for r in runs:
            ever_founds.append(bool(r["ever_found"]))
            iterations.append(r["nfev"])
        # compute p('found minimum')
        #run_info["p_succ"].append(ever_founds)
        run_info["p_succ"].append(
            np.mean(ever_founds, dtype=np.float128 )
        )  # float128 to avoid to many zeros
        run_info["iter"].append(int(math.ceil(np.mean(iterations))))
        # maxiter*shots = actual number of iteration
        run_info["t"].append(r["shots"] * run_info["iter"][-1])
        run_info["shots"].append(r["shots"])
        run_info["qubits"].append(n)
        run_info["circ_depth"].append(p)
        run_info["algo"].append(algo)
        run_info["seed"].append(seed)
        
jobs_df = pd.DataFrame(run_info)
#jobs_df.to_csv(f"data_n{n}_spin_glass.csv")

In [ ]:
jobs_df.describe()
jobs_df.to_csv(f"dataspinglass-n16.csv")

In [ ]:
jobs_df[(jobs_df['circ_depth'] == 2) & (jobs_df['qubits'] == 16)].head()

In [ ]:
all_df = pd.read_csv(f"data_spinglass-ALL.csv", index_col=False)

In [ ]:
complete_df = pd.concat([all_df, jobs_df], ignore_index=True, verify_integrity=True)

In [ ]:
complete_df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
complete_df.to_csv(f"dataspinglass-NEW.csv")

In [ ]:
#fig, (axa, axb, axc) = plt.subplots(1, 3, figsize=(20.4, 5), constrained_layout=True)
for shot, data in jobs_df[jobs_df['algo'] == 'qaoa'].groupby('shots'):
    print(data['p_succ'].groupby('seed'))
    #print(data["p_succ"])
    break
    # plt.xscale("log")
    # plt.title(f"$N=6$ QAOA $p=2$")
    # plt.legend(title='Shots')
#     axa.plot(data["t"], data["p_succ"].apply(np.mean, 1, dtype=np.float128), "-o", label=f"{shot}")
#     axa.plot(data["t"], data["p_succ"].apply(np.mean, 1, dtype=np.float128), "-o", label=f"{shot}")
#     axa.plot(data["t"], data["p_succ"].apply(np.mean, 1, dtype=np.float128), "-o", label=f"{shot}")
    
#     plt.xscale("log")

# axa.title(f"$N={n}$ {algo} $p={data.loc[0, ['circ_depth']]}$")
# axb.title(f"$N={n}$ {algo} $p={data.loc[0, ['circ_depth']]}$")
# axc.title(f"$N={n}$ {algo} $p={data.loc[0, ['circ_depth']]}$")

# plt.legend(title='Shots')

In [ ]:
for _, df in jobs_df.groupby(["shots", "algo"]):
    print(df['p_succ'].mean())


## Comparison $N=14$



In [ ]:
import os

import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt

In [ ]:
n = 14 
circ_depths = np.asarray([1,2,3])
algs = np.asarray(["qaoa", "qaoa+", "vqe"])

model_info = [(n, algo, p) for p in circ_depths for algo in algs]

In [ ]:
run_info = {"t": [], "shots": [], "iter": [], "p_succ": [], "qubits": [], "circ_depth": [], "algo": []}
#for (n, algo, p) in model_info:

for (n, algo, p) in model_info:
    dir_path = f"results/N{n}/{algo}_p{p}/"
    if not os.path.isdir(dir_path):
        print(f"{dir_path} does not exist")
        continue
    for filename in sorted(os.listdir(dir_path)):
        filename = dir_path + filename
        with open(filename, "r") as file:
            # data are a list with #initial_points elements
            runs = json.load(file)
        ever_founds = []
        iterations = [] 
        # for each shot and iteration param
        # we randomized the initial point
        # to estimate the right probability
        for r in runs:
            ever_founds.append(r["ever_found"])
            iterations.append(r["nfev"])
        # compute p('found minimum')
        #run_info["p_succ"].append(ever_founds)
        run_info["p_succ"].append(
            np.mean(ever_founds, dtype=np.float128 )
        )  # float128 to avoid to many zeros
        run_info["iter"].append(np.mean(iterations))
        # maxiter*shots = actual number of iteration
        run_info["t"].append(r["shots"] * run_info["iter"][-1])
        run_info["shots"].append(r["shots"])
        run_info["qubits"].append(n)
        run_info["circ_depth"].append(p)
        run_info["algo"].append(algo)
        
jobs_df = pd.DataFrame(run_info)
#jobs_df.to_csv(f"data_n{n}_spin_glass.csv")

In [ ]:
jobs_df.head()

In [ ]:
for shot, data in jobs_df[(jobs_df['circ_depth'] == 1) & (jobs_df['algo'] == 'vqe')].groupby("shots"):
    plt.plot(data["t"], data["p_succ"], "-o", label=f"{shot}")
    plt.plot()
    plt.xscale("log", base=2)

x_axes = np.arange(0, 2**(3+n))
plt.plot(x_axes, 1 - (1-1/(2**n))**x_axes, '--', color="black", label="Random sampling")
plt.title(f"$N=14$ VQE $p=1$")
plt.legend(title='Shots')

In [ ]:
for shot, data in jobs_df[(jobs_df['circ_depth'] == 2) & (jobs_df['algo'] == 'vqe')].groupby("shots"):
    plt.plot(data["t"]*2, data["p_succ"], "-o", label=f"{shot}")
    plt.plot()
    plt.xscale("log", base=2)

x_axes = np.arange(0, 2**(3+n))
plt.plot(x_axes, 1 - (1-1/(2**n))**x_axes, '--', color="black", label="Random sampling")
plt.title(f"$N=14$ VQE $p=2$")
plt.legend(title='Shots')

In [ ]:
for shot, data in jobs_df[(jobs_df['circ_depth'] == 2) & (jobs_df['algo'] == 'qaoa')].groupby("shots"):
    plt.plot(data["t"]*2, data["p_succ"], "-o", label=f"{shot}")
    plt.plot()
    plt.xscale("log", base=2)

x_axes = np.arange(0, 2**(3+n))
plt.plot(x_axes, 1 - (1-1/(2**n))**x_axes, '--', color="black", label="Random sampling")
plt.title(f"$N=14$ QAOA $p=2$")
plt.legend(title='Shots')

In [ ]:
for shot, data in jobs_df[(jobs_df['circ_depth'] == 2) & (jobs_df['algo'] == 'qaoa+')].groupby("shots"):
    plt.plot(data["t"]*2, data["p_succ"], "-o", label=f"{shot}")
    plt.plot()
    plt.xscale("log", base=2)

x_axes = np.arange(0, 2**(3+n))
plt.plot(x_axes, 1 - (1-1/(2**n))**x_axes, '--', color="black", label="Random sampling")
plt.title(f"$N=14$ QAOA+ $p=2$")
plt.legend(title='Shots')

In [ ]:
jobs_df[jobs_df["algo"] == "qaoa+"].head(20)


# TODO

- [x] nel plot dello scaling esponenziale della prob di successo inserire anche altre famiglie di p, i. e., p=[0.5, 0.6, 0.7, 0.8, 0.9]
- [x] studiare i rumori mediani degli ultimi device di IBM e capire quali sono i valori, a quel punto inserire il rumore nel codice o con una classe prefissata capendo come estrarre i valori del rumore oppure settandoli a mano
- [x] verificare la differenza tra i risultati del circuito rumoroso e quello del simulatore esatto per una taglia ragionevole e stampare i risultati per Guglielmo
- [x] $p=2$ servono 10k per alcuni circuiti, da 14 in poi. VALUTARE BENE!! (non serve lo scaling si vede)
- [x] d'ora in poi i tempi della cpu vanno normalizzati per numero di cpu e mediati per le iterazioni, l'obiettivo è avere un t/CPU medio
- [x] provare modello frustrato $\pm1$ (scritto ma ancora non ho fatto girare)
- [x] inizializzare con hadamart H gate su QAOA-inspired
- [ ] plot in cui faccio vedere lo shape dell'energia e, sopra, anche alcuni passi dell'ottimizzatore come istogramma degli output, così da visualizzare che se non becca il ground state all'inizio ha poche speranze di beccarlo alla fine.  
- [x] QAOA $p=2$ pochi shots più iterazioni (N=14,16,18)
- [ ] Spin glass (QUANTE REALIZZAZIONI di $J_{ij}$ 100?) solo su size $N=14, 16$ e VQE, QAOA $p=2$, e QAOA+. OBC connect.
- [ ] Spin glass con campo locale random $h_i$ connectivity [?]. 
- [ ] Figure: fig1 shape E vs x e diversi circuiti; fig2 vqe vs qaoa $p=2$ con $F(success)$ due panel; fig3 scaling VQE (con e senza rumore) e QAOA $p=2$ due panel; fig4 spin glass
- è



## Test QAOA exact


In [ ]:
import functools

import numpy as np
import qiskit
import qiskit_aer
from scipy import sparse

from src import utils

In [ ]:
qubits = 4
dim = 1
type_ising = 'ferro'
h_field = -0.05
seed = 42
verbose = 1
circ_depth = 1
type_ansatz = 'qaoa'
initial_points = [1]
backend = "statevector"
measure = False

In [ ]:
# hamiltonian is defined with +
# following http://spinglass.uni-bonn.de/ notation
ising, global_min = utils.create_ising1d(qubits, dim, type_ising, h_field, seed)
print(ising)
print(f"J: {ising.adj_dict}\nh: {ising.h_field}\n")
# create the quantum ansatz
ansatz = utils.create_ansatz(
    qubits,
    circ_depth,
    type_ansatz,
    ising,
    measure,
    verbose=verbose,
)
# define generator for initial points
rng = np.random.default_rng(seed=seed)
# define different starting points
if len(initial_points) == 1:
    initial_points.insert(0, 0)
thetas0 = utils.get_init_points(initial_points, ansatz.num_parameters, rng)[0]

def kron(gate_lst):
    return functools.reduce(np.kron, gate_lst)

def pauli_z(i, n):
    if i < 0 or i >= n or n < 1:
        raise ValueError("Bad value of i and/or n.")
    pauli_z_lst = [
        np.array([[1, 0], [0, -1]]) if j == i else np.eye(2) for j in range(n)
    ]
    return kron(pauli_z_lst)

q_hamiltonian = sparse.coo_array(
    sum(
        [
            ising.adj_matrix[i, j]
            * pauli_z(i, ising.spins)
            @ pauli_z(j, ising.spins)
            for i in range(ising.spins)
            for j in range(ising.spins)
        ]
    )
    + sum([ising.h_field[i] * pauli_z(i, ising.spins) for i in range(ising.spins)])
)

In [ ]:
simulator = qiskit_aer.AerSimulator(method=backend)

def update_ansatz(ansatz, parameters: np.ndarray) -> qiskit.QuantumCircuit:
    # assign current thetas to the parametric circuit
    # in_place=False to not overwrite the parametric circuit
    circuit = ansatz.assign_parameters(parameters)
    return circuit

circuit = update_ansatz(ansatz, thetas0)
circuit.save_statevector()

circuit = qiskit.transpile(circuit, simulator)

job = simulator.run(circuit)
# Grab results from the job
result = job.result()
result_statevector = result.get_statevector(circuit).data

In [ ]:
a = q_hamiltonian @ result_statevector

In [ ]:
q_hamiltonian.dot(result_statevector).conjugate().dot(result_statevector)

In [ ]:
(np.real(result_statevector)**2 + np.imag(result_statevector)**2).sum()

In [ ]:
np.vdot(result_statevector,result_statevector)

## Exact simulation
No shot noise


### Test qiskit implementation


In [ ]:
import numpy as np
import qiskit
import qiskit_aer

from src import utils, vqe
from src import ising

In [ ]:
def create_qaoa_ansatz(
    num_qubits: int,
    circ_depth: int,
    hamiltonian: ising.Ising,
    increase_params: bool = False,
) -> qiskit.QuantumCircuit:
    """Create a quantum ansatz inspired by the problem Hamiltonian.
    Setting increase_params=False the ansatz becomes the typical one of QAOA, with 2*circ_depth parameters.
    Otherwise, increase_params=True increases the parameters to (2*num_qubits - 1)*circ_depth.
    See also https://qiskit.org/textbook/ch-applications/qaoa.html.

    Args:
        num_qubits (int): Number of qubits.
        circ_depth (int): Depth of the circuit, minimum 1.
        hamiltonian (ising.Ising): Problem Hamiltonian.
        increase_params (bool, optional): Allows betas and gammas to change in each layer. Defaults to False.

    Returns:
        qiskit.QuantumCircuit: QAOA or QAOA+ ansatz.
    """
    # define circuit
    name = f"QAOA+ p={circ_depth}" if increase_params else f"QAOA p={circ_depth}"
    qc = qiskit.QuantumCircuit(num_qubits, name=name)
    # set initial entalgled state
    for i in range(num_qubits):
        qc.h(i)
    if increase_params:
        # create the parameters for the circuit
        thetas = qiskit.circuit.ParameterVector(
            "$\\theta$", (2 * num_qubits) * circ_depth
        )
    else:
        # calssical QAOA ansatz
        betas = qiskit.circuit.ParameterVector("$\\beta$", circ_depth)
        gammas = qiskit.circuit.ParameterVector("$\\gamma$", circ_depth)
    # add circ_depth layers
    for i in range(circ_depth):
        #add R_zz parametric gates
        for j, j_coupling in enumerate(hamiltonian.adj_dict.values()):
            if increase_params:
                qc.rzz(
                    j_coupling * 2*thetas[(2 * num_qubits) * i + j],
                    j,
                    (j + 1) % hamiltonian.spins,
                )
            else:
                qc.rzz(j_coupling * 2 * gammas[i], j, (j + 1) % hamiltonian.spins)
        qc.barrier()
        #add Rz parametric gates
        for j, h_j in enumerate(hamiltonian.h_field):
            if increase_params:
                qc.rz(h_j * 2 * thetas[(2 * num_qubits) * i + num_qubits - 1 + j], j)
            else:
                qc.rz(h_j * 2 * gammas[i], j)
        # add Rx parametric gates
        for j in range(num_qubits):
            if increase_params:
                qc.rx(-2 * thetas[(2 * num_qubits) * (i + 1) - 1], j)
            else:
                qc.rx(-2 * betas[i], j)
        # do not put barrier in the last iteration
        if i == circ_depth - 1:
            continue
    return qc


def create_ansatz(
    qubits: int,
    circ_depth: int,
    ansatz_type: str,
    hamiltonian: ising.Ising,
    measure: bool,
    verbose: int = 0,
):
    if ansatz_type == "vqe":
        # standard VQE ansatz
        qc = qiskit.circuit.library.RealAmplitudes(
            qubits,
            reps=circ_depth,
            insert_barriers=True,
            entanglement="circular",
            name=f"VQE p={circ_depth}",
        )
    elif ansatz_type == "qaoa" or ansatz_type == "qaoa+":
        increase_params = True if ansatz_type == "qaoa+" else False
        qc = create_qaoa_ansatz(qubits, circ_depth, hamiltonian, increase_params)
    else:
        raise NotImplementedError(f"Ansatz type {ansatz_type} not found")
    if measure:
        qc.barrier()
        qc.measure_all()
    if verbose > 0 and qubits < 8:
        print(qc)
    return qc


def update_ansatz(ansatz, parameters: np.ndarray) -> qiskit.QuantumCircuit:
    # assign current thetas to the parametric circuit
    # in_place=False to not overwrite the parametric circuit
    circuit = ansatz.assign_parameters(parameters)
    return circuit

In [ ]:
def create_ising1d(
    spins: int,
    dim: int,
    type_ising: str,
    h_field: float,
    seed: int,
):
    # hamiltonian is defined with +
    # following http://spinglass.uni-bonn.de/ notation
    # and D-Wave https://docs.dwavesys.com/docs/latest/c_gs_2.html#underlying-quantum-physics
    J, h = utils.get_ising_params(spins, h_field, type_ising, seed)
    adj_dict = {}
    for i in range(spins):
        if i == spins - 1:
            continue
        adj_dict[(i, (i + 1) % spins)] = J[i]
    # class devoted to set the couplings and get the energy
    model = ising.Ising(spins, dim=dim, adj_dict=adj_dict, h_field=h)
    # exact enumeration would be too expensive for large sizes
    if spins < 22 and (type_ising == "binary" or type_ising == "random"):
        min_eng = utils.compute_ising_min(model)
    elif type_ising == "ferro":
        min_eng = model.energy(-np.ones(model.spins))
    else:
        min_eng = -np.inf
    return model, min_eng

In [ ]:
qubits = 3
dim = 1
type_ising = 'ferro'
h_field = 0.05
seed = 42

circ_depth = 1
type_ansatz = 'qaoa'
verbose = 1
initial_points = [1]
shots = [None]
backend = 'automatic'
measure = False

In [ ]:
# hamiltonian is defined with +
# following http://spinglass.uni-bonn.de/ notation
hamiltonian, global_min = create_ising1d(qubits, dim, type_ising, h_field, seed)
print(hamiltonian)
print(f"J: {hamiltonian.adj_dict}\nh: {hamiltonian.h_field}\n")
# create the quantum ansatz
ansatz = create_ansatz(
    qubits,
    circ_depth,
    type_ansatz,
    hamiltonian,
    measure=measure,
    verbose=verbose,
)
# define generator for initial points
rng = np.random.default_rng(seed=seed)
# define different starting points
if len(initial_points) == 1:
    initial_points.insert(0, 0)
#thetas0 = utils.get_init_points(initial_points, ansatz.num_parameters, rng)[0]
thetas0 = [1, 1]

simulator = qiskit_aer.AerSimulator(method=backend)

circuit = update_ansatz(ansatz, thetas0)
circuit.save_statevector()

circuit = qiskit.transpile(circuit, simulator)

job = simulator.run(circuit)
# Grab results from the job
result = job.result()
statevector = result.get_statevector(circuit)

print(statevector)


### Simulation with Matrix

Using qutip and Mazzola's code


In [ ]:
import qutip
from qutip import qobj

In [ ]:
def my_kron(arr_list):
    """Takes a list of numpy arrays [A1, A2, ...] and computes their tensor product A1 (x) A2 (x) ..."""
    if len(arr_list) == 1:
        return arr_list[0]
    else:
        return np.kron(arr_list[0], my_kron(arr_list[1:]))
def X(i, n):
    if i<0 or i>=n or n<1:
        raise ValueError('Bad value of i and/or n.')
    X_list = [np.array([[0,1],[1,0]]) if j==i else np.eye(2) for j in range(n)]
    return my_kron(X_list)
def Y(i, n):
    if i<0 or i>=n or n<1:
        raise ValueError('Bad value of i and/or n.')
    X_list = [np.array([[0,-1j],[1j,0]]) if j==i else np.eye(2) for j in range(n)]
    return my_kron(X_list)
def Z(i, n):
    if i<0 or i>=n or n<1:
        raise ValueError('Bad value of i and/or n.')
    X_list = [np.array([[1,0],[0,-1]]) if j==i else np.eye(2) for j in range(n)]
    return my_kron(X_list)

In [ ]:
def ansatz_matrixform_expectation(thetas, n_qubits, h_prob, h_mix, psi_in, n_blocks):
    # initialize the circuit operations (in matrix form) - make the ansatz
    thetas_counter = 0
    psi_circuit = psi_in
    for block in range(n_blocks):
        temp = -1.j*(-1)*thetas[thetas_counter]*h_prob
        psi_circuit = temp.expm()*psi_circuit
        thetas_counter += 1
        temp = -1.j*(-1)*thetas[thetas_counter]*h_mix
        psi_circuit = temp.expm()*psi_circuit
        thetas_counter += 1
    # measure the expectation value
    val = qutip.expect(-1*h_prob, psi_circuit)
    
    #print("C", val)
    
    return val , psi_circuit

def initstate_Had(L):
  psi_initial = qutip.basis(2,0)+qutip.basis(2,1)
  for i in range(L-1):
      psi_initial = qutip.tensor(psi_initial, qutip.basis(2,0)+qutip.basis(2,1))
  return psi_initial/psi_initial.norm()


In [ ]:
Llist = [3]

for L in Llist:
    Mshots=16
    Gamma=0
    Jcoupl=1
    h_Q = np.zeros(L)-0.05*np.ones(L)
    J_Q = np.diag(np.ones(L-1), k=1)
    #J_Q[0,L-1]=1 #PBC
    LambdaReg=L+1
    H_z = sum([J_Q[i,j]*Z(i,L) @ Z(j,L) for i in range(L) for j in range(L)]) + sum([h_Q[i]*Z(i,L) for i in range(L)])
    H_x = sum([X(i,L) for i in range(L)])

    #print(H_z)
    H = -Gamma*H_x-Jcoupl*H_z

    dims = [[2] * L, [2] * L]
    mazzola_hamiltonian = qobj.Qobj(H, dims=dims)
    h_prob = qobj.Qobj(H_z, dims=dims)
    #print(h_prob.expm())
    h_mix = qobj.Qobj(H_x, dims=dims)

    thetas=[1,1] #[6.15816242, 2.378081]
    n_blocks=1 

    psi_in = initstate_Had(L)

    value, psi = ansatz_matrixform_expectation(thetas, L, h_prob, h_mix, psi_in,n_blocks)

In [ ]:
print(f"Guglielmo's loss {value}")
print(f"Guglielmo's psi \n {psi.data}")

In [ ]:
loss = np.real(
    hamiltonian.quantum_hamiltonian.dot(statevector)
    .conjugate()
    .dot(statevector)
)
print(f"my loss {loss}")
print(f"my psi \n{statevector.data[::-1]}")

In [ ]:
print(f"hamiltonian: {thetas} circuit:{thetas0}")